In [ ]:
"""
Hi Humble

This is an example notebook showing how to load the data, save it as a CSV, or load it as a dataframe

I've copied this  code from the "run.py" file next to me. I also have some extra comments

"""

In [ ]:
import os
import logging
import pandas as pd
import datetime
import numpy as np
import math

#The "functions" directory has all the specific logic. Your notebook needs to be in this directory to access the functions directory.
from functions import well_data as well_data_functions
from functions import baro_data as baro_data_functions

from functions import dataframes as dataframe_functions

#This prints the logging messages. Comment it out to set it to silent. Change to default='DEBUG' to get much more messages.
logging.basicConfig(level=(os.getenv('LOG_LEVEL', default='INFO')) )

#Change the paths here if your input data is in another directory. Note, ".." means "go up a directory"
raw_well_data_directory =  os.path.relpath('../rawdata/Welldata/')
raw_baro_data_directory = os.path.relpath('../rawdata/Baro/')
out_directory =  os.path.relpath('out')

#Run functions to handle baro data
baro_dataframe = baro_data_functions.create_baro_dataframes(raw_baro_data_directory)

#Run functions to handle well data
well_dataframes = well_data_functions.create_dataframes_for_all_wells(raw_well_data_directory)
well_dataframe = dataframe_functions.merge_dataframes(well_dataframes)

#Merge the dataframes together
df = dataframe_functions.merge_dataframes([baro_dataframe, well_dataframe])
logging.info("Merged all dataframes into a single time series dataframe")

#And writes it to disk if we want to write it to disk.
# dataframe_functions.dataframe_to_csv(df, out_directory, 'output.csv')

In [ ]:
Info = pd.read_csv('../Info.csv', delimiter = ',')
# Set well as the index
Info.set_index('well', inplace = True)

In [ ]:
K_dist_river = pd.read_csv('../K_dist_river.csv', delimiter = ',')
# Set well as the index
K_dist_river.set_index('well', inplace = True)

In [ ]:
Res_SD = pd.read_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\Res_SD_results.csv', delimiter = ',')
Res_SD.set_index('Well', inplace = True)

In [ ]:
# correction for wells that have original conductivity data in uS/cm
df["APP192_cond_uScm"] = df["APP192_cond_uScm"]/1000
#df["BHBUR27_cond_uScm"] = df["BHBUR27_cond_uScm"]/1000
df["APP103_cond_uScm"] = df["APP103_cond_uScm"]/1000
df["APP165_cond_uScm"] = df["APP165_cond_uScm"]/1000

In [ ]:
# Delete erroneous readings

my_wells = []
for col in df.columns:
    wellname = col.split("_")[0]
    my_wells.append(wellname)
my_wells=list(dict.fromkeys(my_wells).keys())
    
for well in my_wells:
    for col in df.columns:
        wellname = col.split("_")[0]
        
        if wellname == "APP192":
            df.at[datetime.datetime(2021, 1, 18, 13, 20), col] = None
            df.at[datetime.datetime(2021, 4, 16, 10, 20), col] = None

        if wellname == "OwlesTce":
            df.at[datetime.datetime(2021, 4, 16, 14, 10), col] = None
            df.at[datetime.datetime(2021, 4, 16, 14, 20), col] = None
            df.at[datetime.datetime(2021, 6, 10, 14, 0), col] = None
            df.at[datetime.datetime(2020, 11, 25, 9, 10), col] = None
    
        if wellname == "ParengaPl":
            df.at[datetime.datetime(2020, 11, 25, 9, 50), col] = None
            df.at[datetime.datetime(2021, 1, 18, 11, 20), col] = None
            df.at[datetime.datetime(2021, 6, 10, 11, 10), col] = None
        
        if wellname == "MarigoldLn":
            df.at[datetime.datetime(2020, 10, 14, 12, 50), col] = None
            df.at[datetime.datetime(2021, 1, 18, 10, 40), col] = None
            df.at[datetime.datetime(2021, 1, 18, 10, 50), col] = None
            df.at[datetime.datetime(2021, 4, 16, 11, 0), col] = None
            df.at[datetime.datetime(2021, 4, 16, 11, 40), col] = None
            df.at[datetime.datetime(2021, 4, 16, 11, 50), col] = None
        
        if wellname == "EvansAve":
            df.at[datetime.datetime(2021, 1, 18, 12, 10), col] = None
            df.at[datetime.datetime(2021, 4, 16, 14, 30), col] = None
            df.at[datetime.datetime(2020, 10, 14, 10, 50), col] = None
            df.at[datetime.datetime(2020, 10, 14, 11, 0), col] = None
            df.at[datetime.datetime(2020, 10, 14, 11, 10), col] = None
            df.at[datetime.datetime(2020, 11, 25, 8, 30), col] = None

In [ ]:
    wells = os.listdir(raw_well_data_directory)
    well_dataframes = []
    for well in wells:
        
        # Calculation for density based on Langevin et al. (2008)
        df[f"{well}_density_kgm3"] = 1000+0.7*0.68*0.001*df[f"{well}_cond_uScm"]
        # Calculation for actual water column height (m)
        df[f"{well}_wat_col_m"] =(df[f"{well}_total_GW_pressure_kPa"]-df["baro_pressure_kPa"])*1000/(df[f"{well}_density_kgm3"]*9.80665)
        # Calculation for depth to WL from TOC
        df[f"{well}_WL_TOC_m"] = Info.loc[f"{well}", 'TOC_ZeroPoint_m'] - df[f"{well}_wat_col_m"]
        # Calculation for depth to WL from GL
        df[f"{well}_WL_GL_m"] = Info.loc[f"{well}", 'GL_ZeroPoint_m'] - df[f"{well}_wat_col_m"]
        # Calculation for WL from Lyt (hydraulic head)
        df[f"{well}_WL_Lyt_m"] = Info.loc[f"{well}", 'TOC_Lyt_m'] - df[f"{well}_WL_TOC_m"]
        # Calculation for freshwater head or hf (m)
        # hf = pa*h/pf - (pa-pf)*z/pf
        # Assuming that the the density measured by logger is the average density
        df[f"{well}_hf_Lyt_m"] = df[f"{well}_density_kgm3"]*df[f"{well}_WL_Lyt_m"]/1000 - (df[f"{well}_density_kgm3"] - 1000) * Info.loc[f"{well}", 'z'] / 1000


In [ ]:
# Processing river salinity Odyssey data

# Read River csv file
River_salinity_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettySalinity_brick\OwlesJettySalinity_20201219_20210118.csv'
River_salinity = pd.read_csv(River_salinity_CSV_path, delimiter = ',', skiprows=[0,1,2,3,4,5,7,8])

# Rename columns
River_salinity = River_salinity.rename(columns={"Date and Time":"date", "       Temperature":"time", "Salinity":"river_odyssey_temp", "Unnamed: 6":"river_cond_mScm",})

# Drop columns
River_salinity = River_salinity.drop(["        ", "Scan No ", "        .1"], axis=1)

# Replace 24:00:00 with 00:00:00
River_salinity['time'] = River_salinity['time'].replace([' 24:00:00 '],' 00:00:00 ')

# Make Python recognize the date column as a datetime object
River_salinity['date']=pd.to_datetime(River_salinity['date'], format = '       %d/%m/%Y ').dt.date

# Import datetime
import datetime

# Correction of wrong date at midnight
# If time = 00:00:00, add date by 1 day
River_salinity.loc[River_salinity['time'] == ' 00:00:00 ', 'date2'] = River_salinity['date'] + datetime.timedelta(days=1)
River_salinity.loc[River_salinity['time'] != ' 00:00:00 ', 'date2'] = River_salinity['date']

# Make Python recognize time column as datetime object
River_salinity['time']=pd.to_datetime(River_salinity['time'], format = ' %H:%M:%S ').dt.time
# Why does the time column have 1900 date??
# Adding data type time (dt.time) at the end made it work!

# Combine columns date2 and time
River_salinity.loc[:,'date_time'] = pd.to_datetime(River_salinity.date2.astype(str)+' '+River_salinity.time.astype(str))
# I don't know how this works but it does! Btw it's already in datetime object format!

# Remove date and time columns in Marigold_4
River_salinity = River_salinity.drop(["date", "time", "date2"], axis=1)

# Set date_time as the index
River_salinity.set_index('date_time', inplace = True)

# Import datetime
from datetime import datetime

# Delete individual erroneous readings
lines_to_drop = [
    datetime(2020, 12, 19, 16, 10),
    datetime(2021, 1, 18, 15, 40),
    datetime(2021, 1, 18, 15, 50)
]

for line in lines_to_drop:
    River_salinity = River_salinity.drop(line)
    
# Processing river salinity Odyssey data

# Read River csv file
River_salinity2_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettySalinity_brick\OwlesJettySalinity_20210118_20210416.csv'
River_salinity2 = pd.read_csv(River_salinity2_CSV_path, delimiter = ',', skiprows=[0,1,2,3,4,5,6,7,8,10,11])

# Rename columns
River_salinity2 = River_salinity2.rename(columns={"Date":"date", "Time":"time", "        ": "river_odyssey_temp", "        .1":"river_cond_mScm"})

# Drop columns
River_salinity2 = River_salinity2.drop(["Salinity", "Scan No ", "       Temperature", "Unnamed: 7"], axis=1)

# Replace 24:00:00 with 00:00:00
River_salinity2['time'] = River_salinity2['time'].replace([' 24:00:00 '],' 00:00:00 ')

# Make Python recognize the date column as a datetime object
River_salinity2['date']=pd.to_datetime(River_salinity2['date'], format = '       %d/%m/%Y ').dt.date

# Import datetime
import datetime

# Correction of wrong date at midnight
# If time = 00:00:00, add date by 1 day
River_salinity2.loc[River_salinity2['time'] == ' 00:00:00 ', 'date2'] = River_salinity2['date'] + datetime.timedelta(days=1)
River_salinity2.loc[River_salinity2['time'] != ' 00:00:00 ', 'date2'] = River_salinity2['date']

# Make Python recognize time column as datetime object
River_salinity2['time']=pd.to_datetime(River_salinity2['time'], format = ' %H:%M:%S ').dt.time
# Why does the time column have 1900 date??
# Adding data type time (dt.time) at the end made it work!

# Combine columns date2 and time
River_salinity2.loc[:,'date_time'] = pd.to_datetime(River_salinity2.date2.astype(str)+' '+River_salinity2.time.astype(str))
# I don't know how this works but it does! Btw it's already in datetime object format!

# Remove date and time columns in Marigold_4
River_salinity2 = River_salinity2.drop(["date", "time", "date2"], axis=1)

# Set date_time as the index
River_salinity2.set_index('date_time', inplace = True)

# Import datetime
from datetime import datetime

# Delete individual erroneous readings
lines_to_drop = [
    datetime(2021, 1, 18, 16, 0),
    datetime(2021, 1, 18, 16, 10),
    datetime(2021, 4, 16, 15, 10)
]

for line in lines_to_drop:
    River_salinity2 = River_salinity2.drop(line)
    
# Processing river salinity Odyssey data

# Read River csv file
River_salinity3_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettySalinity_brick\OwlesJettySalinity_20210416_20210610.csv'
River_salinity3 = pd.read_csv(River_salinity3_CSV_path, delimiter = ',', skiprows=[0,1,2,3,4,5,6,7,8,10,11])

# Rename columns
River_salinity3 = River_salinity3.rename(columns={"Date":"date", "Time":"time", "        ": "river_odyssey_temp", "        .1":"river_cond_mScm"})

# Drop columns
River_salinity3 = River_salinity3.drop(["Salinity", "Scan No ", "       Temperature", "Unnamed: 7"], axis=1)

# Replace 24:00:00 with 00:00:00
River_salinity3['time'] = River_salinity3['time'].replace([' 24:00:00 '],' 00:00:00 ')

# Make Python recognize the date column as a datetime object
River_salinity3['date']=pd.to_datetime(River_salinity3['date'], format = '       %d/%m/%Y ').dt.date

# Import datetime
import datetime

# Correction of wrong date at midnight
# If time = 00:00:00, add date by 1 day
River_salinity3.loc[River_salinity3['time'] == ' 00:00:00 ', 'date2'] = River_salinity3['date'] + datetime.timedelta(days=1)
River_salinity3.loc[River_salinity3['time'] != ' 00:00:00 ', 'date2'] = River_salinity3['date']

# Make Python recognize time column as datetime object
River_salinity3['time']=pd.to_datetime(River_salinity3['time'], format = ' %H:%M:%S ').dt.time
# Why does the time column have 1900 date??
# Adding data type time (dt.time) at the end made it work!

# Combine columns date2 and time
River_salinity3.loc[:,'date_time'] = pd.to_datetime(River_salinity3.date2.astype(str)+' '+River_salinity3.time.astype(str))
# I don't know how this works but it does! Btw it's already in datetime object format!

# Remove date and time columns in Marigold_4
River_salinity3 = River_salinity3.drop(["date", "time", "date2"], axis=1)

# Set date_time as the index
River_salinity3.set_index('date_time', inplace = True)

# Import datetime
from datetime import datetime

# Delete individual erroneous readings
lines_to_drop = [
    datetime(2021, 6, 10, 12, 10)
]

for line in lines_to_drop:
    River_salinity3 = River_salinity3.drop(line)
    
# Processing river salinity Odyssey data

# Read River csv file
River_salinity4_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettySalinity_brick\OwlesJettySalinity_20210610_20210922.csv'
River_salinity4 = pd.read_csv(River_salinity4_CSV_path, delimiter = ',', skiprows=[0,1,2,3,4,5,6,7,8,10,11])

# Rename columns
River_salinity4 = River_salinity4.rename(columns={"Date":"date", "Time":"time", "        ": "river_odyssey_temp", "        .1":"river_cond_mScm"})

# Drop columns
River_salinity4 = River_salinity4.drop(["Salinity", "Scan No ", "       Temperature"], axis=1)

# Replace 24:00:00 with 00:00:00
River_salinity4['time'] = River_salinity4['time'].replace([' 24:00:00 '],' 00:00:00 ')

# Make Python recognize the date column as a datetime object
River_salinity4['date']=pd.to_datetime(River_salinity4['date'], format = '       %d/%m/%Y ').dt.date

# Import datetime
import datetime

# Correction of wrong date at midnight
# If time = 00:00:00, add date by 1 day
River_salinity4.loc[River_salinity4['time'] == ' 00:00:00 ', 'date2'] = River_salinity4['date'] + datetime.timedelta(days=1)
River_salinity4.loc[River_salinity4['time'] != ' 00:00:00 ', 'date2'] = River_salinity4['date']

# Make Python recognize time column as datetime object
River_salinity4['time']=pd.to_datetime(River_salinity4['time'], format = ' %H:%M:%S ').dt.time
# Why does the time column have 1900 date??
# Adding data type time (dt.time) at the end made it work!

# Combine columns date2 and time
River_salinity4.loc[:,'date_time'] = pd.to_datetime(River_salinity4.date2.astype(str)+' '+River_salinity4.time.astype(str))
# I don't know how this works but it does! Btw it's already in datetime object format!

# Remove date and time columns in Marigold_4
River_salinity4 = River_salinity4.drop(["date", "time", "date2"], axis=1)

# Set date_time as the index
River_salinity4.set_index('date_time', inplace = True)

# Import datetime
from datetime import datetime

# Delete individual erroneous readings
lines_to_drop = [
    datetime(2021, 9, 22, 12, 50)
]

for line in lines_to_drop:
    River_salinity4 = River_salinity4.drop(line)
    
River_salinity_concat = pd.concat([River_salinity, River_salinity2, River_salinity3, River_salinity4])

del River_salinity, River_salinity2, River_salinity3, River_salinity4, River_salinity_CSV_path, River_salinity2_CSV_path, River_salinity3_CSV_path, River_salinity4_CSV_path

In [ ]:
# Processing river level HOBO data

# Read River csv file
River_level_a_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettyLevel_20200617_20201014.csv'
River_level_a = pd.read_csv(River_level_a_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level_a = River_level_a.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 20183528, SEN S/N: 20183528, LBL: Abs Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 20183528, SEN S/N: 20183528, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level_a = River_level_a.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level_a['date_time']=pd.to_datetime(River_level_a['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level_a.set_index('date_time', inplace = True)

# Read River csv file
River_level_b_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettyLevel_20201014_20201016.csv'
River_level_b = pd.read_csv(River_level_b_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level_b = River_level_b.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 20183528, SEN S/N: 20183528, LBL: Abs Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 20183528, SEN S/N: 20183528, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level_b = River_level_b.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level_b['date_time']=pd.to_datetime(River_level_b['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level_b.set_index('date_time', inplace = True)


# Read River csv file
River_level2_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettylevel_20201210_20210118.csv'
River_level2 = pd.read_csv(River_level2_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level2 = River_level2.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Absolute Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level2 = River_level2.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level2['date_time']=pd.to_datetime(River_level2['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level2.set_index('date_time', inplace = True)

# No readings were deleted after manual checks of head & tail readings - they seem OK

# Processing river level HOBO data

# Read River csv file
River_level3_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettyLevel_20210118_20210416.csv'
River_level3 = pd.read_csv(River_level3_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level3 = River_level3.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Absolute Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level3 = River_level3.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level3['date_time']=pd.to_datetime(River_level3['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level3.set_index('date_time', inplace = True)

# Import datetime
from datetime import datetime

# Delete individual erroneous readings
lines_to_drop = [
    datetime(2021, 4, 16, 15, 0)
]

for line in lines_to_drop:
    River_level3 = River_level3.drop(line)
    
# Processing river level HOBO data

# Read River csv file
River_level4_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettyLevel_20210416_20210610.csv'
River_level4 = pd.read_csv(River_level4_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level4 = River_level4.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Absolute Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level4 = River_level4.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level4['date_time']=pd.to_datetime(River_level4['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level4.set_index('date_time', inplace = True)

# No readings were deleted after manual checks of head & tail readings - they seem OK

# Processing river level HOBO data

# Read River csv file
River_level5_CSV_path = r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\rawdata\River\OwlesJettyLevel\OwlesJettyLevel_20210610_20210922.csv'
River_level5 = pd.read_csv(River_level5_CSV_path, delimiter = ',', skiprows=[0])

# Rename columns
River_level5 = River_level5.rename(columns={
    "Date Time, GMT+12:00":"date_time",
    "Abs Pres, kPa (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Absolute Pressure)":"total_river_pressure_kPa",
    "Temp, °C (LGR S/N: 10808243, SEN S/N: 10808243, LBL: Temperature)":"river_HOBO_temp"
    })

# Drop "#" column
River_level5 = River_level5.drop(["#"], axis=1)

# Make Python recognize the date_time column as a datetime object
River_level5['date_time']=pd.to_datetime(River_level5['date_time'], format = '%m/%d/%y %I:%M:%S %p')

# Set date_time as the index
River_level5.set_index('date_time', inplace = True)

# No readings were deleted after manual checks of head & tail readings - they seem OK

# Concatenate df's
River_level_concat = pd.concat([River_level_a, River_level_b, River_level2, River_level3, River_level4, River_level5])

del River_level_a, River_level_b, River_level2, River_level3, River_level4, River_level5, River_level_a_CSV_path, River_level_b_CSV_path, River_level2_CSV_path, River_level3_CSV_path, River_level4_CSV_path, River_level5_CSV_path

In [ ]:
# Merge river_salinity_concat and river_level_concat
River = River_salinity_concat.merge(River_level_concat, how = 'outer', left_on = 'date_time', right_on = 'date_time')

In [ ]:
# Merge River and df
df2 = df.merge(River, how = 'outer', left_on = 'date_time', right_on = 'date_time')
del df, River

In [ ]:
# Read rain csv file
rain = pd.read_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\Rainfall\Hourly_rainfall_BromleyEws43967_20210101_0000_20210929_1500.csv', delimiter = ',')

# Combine the date and time columns in rain
rain.insert(2, 'date_time', rain['date'] + ' ' + rain['time'])

# Make Python recognize the date_time column as a datetime object in the three dataframes
rain['date_time']=pd.to_datetime(rain['date_time'], format = '%d/%m/%Y %H:%S')

# Remove date and time columns in APP165
rain = rain.drop(["date", "time"], axis=1)

# Set date_time as the index of APP165_1
rain.set_index('date_time', inplace = True)

df2 = df2.merge(rain, how = 'outer', left_on = 'date_time', right_on = 'date_time')

del rain

# Calculate rain in metres per hour
df2["rain_m"] = df2["rain_mm"]/1000

# Net recharge in metres per hour = 25% of rainfall
df2["Wnet_m"] = 0.25 * df2["rain_m"]

In [ ]:
# Calculation for river density
df2["river_density_kgm3"] = 1000 + 0.7*0.68*df2["river_cond_mScm"]

# Calculation for river head or river water column height above sensor (m)
# h=(p-pb)/(rho x g)

df2["river_wat_col_m"] =(df2["total_river_pressure_kPa"]-df2["baro_pressure_kPa"])*1000/(df2["river_density_kgm3"]*9.80665)

# Calculation for river level from Lyt
# River water level = Top of warratah elevation Lyt in m - (Distance between top of warratah and zero point of WL logger in m- absolute sensor depth in m)
# Elevation of top of warratah as per 22 Sept 2021 using RTK = 0.0865 m Lyttelton datum (see timeseries_inventory.xlsx)
# Elevation of top of warratah as per 19 Dec 2020 survey = -0.001 m Lyttelton datum. The difference between both surveys is within the error of +/- 0.1 m.

df2["river_level_Lyt_m"]=0.0865-(0.542-df2["river_wat_col_m"])

# Q: does the warratah sink over time? I.e. sensor elevation/placement changes too?

# Calculation for river freshwater head or hf (m)
# hf = pa*h/pf - (pa-pf)*z/pf
# Assuming that the the density measured by logger is the average density
# The water depth was recorded as 860 mm on 19/12/20 at 15:54 NZST
# The elevation of the bottom of the river (z) is -0.86 m + water level relative to Lyt at 15:55 NZST, which is 0.038 m = -0.822 m Lyt

df2["river_hf_Lyt_m"] = df2["river_density_kgm3"]*df2["river_level_Lyt_m"]/1000 - (df2["river_density_kgm3"]-1000)*(-0.822)/1000

op = 1
oz = 0.015

oh = 0.055 # River average residual

# Hf error propagation for river
df2["river_a"] = df2["river_density_kgm3"] * df2["river_level_Lyt_m"] / 1000
df2["river_oa"] = np.abs(df2["river_a"]) * np.sqrt( (op/df2["river_density_kgm3"])**2 + (oh/df2["river_level_Lyt_m"])**2 )

df2["river_b"] = (df2["river_density_kgm3"] - 1000)/1000
df2["river_ob"] = np.abs(df2["river_b"]) * np.sqrt((op/(df2["river_density_kgm3"]-1000))**2)

df2["river_c"] = df2["river_b"]*(-0.822)
df2["river_oc"] = np.abs(df2["river_c"]) * np.sqrt( (df2["river_ob"]/df2["river_b"])**2 + (oz/(-0.822))**2)
df2["river_ohf"] = np.sqrt(df2["river_oa"]**2 + df2["river_oc"]**2)

#volume = pd.DataFrame()

wells = os.listdir(raw_well_data_directory)
well_dataframes = []
for well in wells:
    # Calculation for difference between river spec cond and GW spec cond
    df2[f"{well}_dSC_uScm"] = df2["river_cond_mScm"] * 1000 - df2[f"{well}_cond_uScm"]
    
    # Calculation for difference between river temp and GW temp
    df2[f"{well}_dtemp"] = df2["river_odyssey_temp"] - df2[f"{well}_temp"]
    
    # Calculation for difference between river hf and GW hf
    df2[f"{well}_dhf_m"] =  df2[f"{well}_hf_Lyt_m"] - df2["river_hf_Lyt_m"]
    
    # Look up oh from Results table
    oh = Res_SD.loc[f"{well}",'Res_SD']
    
    # Hf error propagation; checked results in Excel out/error_test_ParengaPl_20210120.xlsx
    df2[f"{well}_a"] = df2[f"{well}_density_kgm3"] * df2[f"{well}_WL_Lyt_m"] / 1000
    df2[f"{well}_oa"] = np.abs(df2[f"{well}_a"]) * np.sqrt( (op/df2[f"{well}_density_kgm3"])**2 + (oh/df2[f"{well}_WL_Lyt_m"])**2 )

    df2[f"{well}_b"] = (df2[f"{well}_density_kgm3"] - 1000)/1000
    df2[f"{well}_ob"] = np.abs(df2[f"{well}_b"]) * np.sqrt((op/(df2[f"{well}_density_kgm3"]-1000))**2)

    df2[f"{well}_c"] = df2[f"{well}_b"]*Info.loc[f"{well}", 'z']
    df2[f"{well}_oc"] = np.abs(df2[f"{well}_c"]) * np.sqrt( (df2[f"{well}_ob"]/df2[f"{well}_b"])**2 + (oz/Info.loc[f"{well}", 'z'])**2)
    df2[f"{well}_ohf"] = np.sqrt(df2[f"{well}_oa"]**2 + df2[f"{well}_oc"]**2)
    
    # Horizontal darcy velocity or specific discharge or flux q = -K(h1-h2)/(j1-j2) unit m/day
    df2[f"{well}_river_q_md"] = (-1) * K_dist_river.loc[f"{well}", 'Ave_K_m/d'] * (df2["river_hf_Lyt_m"] - df2[f"{well}_hf_Lyt_m"])/(K_dist_river.loc[f"{well}", 'Dist_river_m'])
    
    # V is the volume of water in m3 displaced [during a tidal cycle or longer] per unit of cross sectional area
    # V=q dt
    df2[f"{well}_vol_m3/m2"] = df2[f"{well}_river_q_md"] *10 / (24*60)
    
    # Flux error propagation
    df2[f"{well}_river_ohf"] = np.sqrt(df2[f"{well}_ohf"]**2 + df2["river_ohf"]**2)
    # Eq 13 in Post et al. (2018) oi/|ij|
    #SD of delta j assumed to be sqrt of 2
    df2[f"{well}_hfx_SD"] = np.sqrt((df2[f"{well}_river_ohf"]/df2[f"{well}_dhf_m"])**2 + (np.sqrt(2)/(K_dist_river.loc[f"{well}", 'Dist_river_m']))**2)
    # SD of horizontal flux qx
    df2[f"{well}_qx_SD"] = df2[f"{well}_river_q_md"]*np.sqrt(((K_dist_river.loc[f"{well}", 'SD_K_m/d'])/(K_dist_river.loc[f"{well}", 'Ave_K_m/d']))**2+(df2[f"{well}_hfx_SD"])**2)
    
    # Hydraulic gradient
    df2[f"{well}_dhf_dx"] = df2[f"{well}_dhf_m"]/K_dist_river.loc[f"{well}", 'Dist_river_m']
    
    # Hydraulic gradient error; oi; See eq. 10 in Revised_manuscript_EndNote.docx or eq. 13 in Post et al. (2018)
    df2[f"{well}_oi"] = np.abs(df2[f"{well}_dhf_dx"])*df2[f"{well}_hfx_SD"]
    
    #d = {'Well': f"{well}", 'Sum_vol_m3/m2':df2[f"{well}_vol_m3/m2"].sum()}
    #volume = volume.append(d, ignore_index=True)
    
    # q0 calculation (units: m2/hour)
    df2[f"{well}_q0"] = (K_dist_river.loc[f"{well}", 'Ave_K_m/h'] * ((df2[f"{well}_hf_Lyt_m"] + K_dist_river.loc[f"{well}", 'z0_m'])**2 - (1 + df2["river_b"]) * K_dist_river.loc[f"{well}", 'z0_m']**2) + df2["Wnet_m"] * K_dist_river.loc[f"{well}", 'Dist_river_m']**2) / (2 * K_dist_river.loc[f"{well}", 'Dist_river_m'])
    
    # xT calculation
    # Zone 1
    # df2[f"{well}_xT_m"] = df2[f"{well}_q0"] / df2["Wnet_m"] - np.sqrt((df2[f"{well}_q0"]/df2["Wnet_m"])**2 - (K_dist_river.loc[f"{well}", 'Dist_river_m'] * df2["river_b"] * (1 + df2["river_b"]) * K_dist_river.loc[f"{well}", 'z0_m']**2)/df2["Wnet_m"])
    # Zone 2
    df2[f"{well}_xT_m"] = ((1 + df2["river_b"]) / df2["river_b"]) * K_dist_river.loc[f"{well}", 'Ave_K_m/h'] * df2[f"{well}_hf_Lyt_m"]**2 / (2 * K_dist_river.loc[f"{well}", 'Dist_river_m']) + (df2["Wnet_m"] * K_dist_river.loc[f"{well}", 'Dist_river_m'] / 2)
    
del op, oh, oz

# Calculation for dhf between APP192 and OwlesTce
df2["APP192_OwlesTce_dhf_m"] = df2["APP192_hf_Lyt_m"] - df2["OwlesTce_hf_Lyt_m"]

# Convert river SC in mS/cm to uS/cm
df2["river_cond_uScm"] = df2["river_cond_mScm"]*1000

In [ ]:
# Vertical flow (specific discharge) between APP192 and OwlesTce
# Vertical component of specific discharge = -K (dhf/dz + (p-pf)/pf)
# K was taken as the average between APP192 and OwlesTce
# vertical distance between mid-point of well screens (dz) is 1.3702 m, assuming straight wells, see logger_info.xlsx
# Need to double check this with RTK measurements!

df2["APP192_OwlesTce_qz"] = ((K_dist_river.loc['OwlesTce', 'Ave_K_m/d'] + K_dist_river.loc['APP192', 'Ave_K_m/d'])/2) *(df2["APP192_OwlesTce_dhf_m"]/1.3702 + ((df2["OwlesTce_density_kgm3"]+df2["APP192_density_kgm3"])/2-1000)/1000)

#df2["APP192_OwlesTce_dhf_m"]/1.3702

In [ ]:
#volume.to_csv(os.path.relpath('../out/Volume_sum_20220413_1140.csv'))
#del volume

In [ ]:
# Read APP207 CSV
APP207 = pd.read_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Data\APP207\BX24_2631_AB.csv', delimiter=',')

# Make Python recognize the date_time column as a datetime object
APP207['date_time']=pd.to_datetime(APP207['date_time'], format = '%d/%m/%Y %H:%M')

# Set date_time as the index
APP207.set_index('date_time', inplace = True)

# Resample to every 5 min
APP207=APP207.resample('5T').asfreq()

# Interpolate NaN values using fifth order polynomial function
APP207=APP207.interpolate(method='polynomial', order=5)

# Resample to every 10 min
APP207=APP207.resample('10T').asfreq()

# Drop NaN rows
APP207=APP207.dropna()

# Merge with df
df2 = df2.merge(APP207, how = 'outer', left_on = 'date_time', right_on = 'date_time')

del APP207

### df3 filters datetime 1 Jan - 29 Sept 2021

In [ ]:
df3=df2.loc['2021-01-01':'2021-06-09']

In [ ]:
df3 = df3.sort_values(by="date_time")

In [ ]:
del df2

### Interpolate NaN values

In [ ]:
# Interpolate NaN values using third order spline function
df3 = df3.interpolate(method='polynomial', order=5)

## Sea level Feb 2021 fourier transform

In [ ]:
from scipy.fft import rfft, rfftfreq

In [ ]:
from scipy import signal as sig

In [ ]:
import datetime
import matplotlib.pyplot as plt

In [ ]:
dot_colour='#c34c54'

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 28)

x = Tide.index

y = Tide['sea_level_m_Lyt']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
#ax.set_ylim(0,1)
ax.set_ylabel("Sea level (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Sea_level_mLVD_202102.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
Tide_Feb=Tide.loc['2021-02-01':'2021-02-28'].to_numpy().flatten()

In [ ]:
np.count_nonzero(Tide_Feb)

In [ ]:
N = np.count_nonzero(Tide_Feb)

fft_output = rfft(Tide_Feb)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/1440)[:N//2] #Sample rate is 1440 points per day (24h/d*60min/h / 1 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.02)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf)

ax.set_ylabel("Amplitude (m)")
ax.set_xlabel("Frequency (cpd)")
ax.set_ylim(0,0.9)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'ro')

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Sea_level_freq_202102.png', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output = output.sort_values('amplitude', ascending=False)

In [ ]:
output

In [ ]:
del Tide_Feb

In [ ]:
# Delete anomalies in tide data
Tide.at[datetime.datetime(2021, 1, 11, 3, 15):datetime.datetime(2021, 1, 11, 3, 17), 'sea_level_m_Lyt'] = None
Tide.at[datetime.datetime(2021, 1, 14, 1, 21):datetime.datetime(2021, 1, 14, 1, 22), 'sea_level_m_Lyt'] = None

In [ ]:
# Interpolate NaN values using third order spline function
Tide = Tide.interpolate(method='polynomial', order=5)

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 5)

x = Tide.index

y = Tide['sea_level_m_Lyt']

fig, (ax1,ax) = plt.subplots(1,2)

fig.subplots_adjust(right=0.75)

fig.suptitle('Sea level',x=0.43)

p1, = ax1.plot(x, y, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(0,1)
ax1.set_ylabel("Sea level (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

del myFmt

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)

ax1.text(datetime.datetime(2021,1,1,2,0), 1.45, "Time domain")

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Sea_level_mLVD_202101_corrected.png', dpi=300, bbox_inches="tight")

del x, y

#-----

N = np.count_nonzero(Tide_Jan)

fft_output = rfft(Tide_Jan)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/1440)[:N//2] #Sample rate is 1440 points per day (24h/d*60min/h / 1 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.02)[0]

fig.subplots_adjust(right=0.75)

fig.set_figwidth(14)
fig.set_figheight(5)

p2, = ax.plot(xf, yf)

ax.set_ylabel("Amplitude (m)")
ax.set_xlabel("Frequency (cpd)")
ax.set_ylim(0,0.9)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color='salmon')

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

ax.text(0.1,0.87,'Frequency domain')

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Sea_level_time_freq_withTitle_202101.png', dpi=300, bbox_inches="tight")

plt.show()



In [ ]:
Tide_Jan=Tide.loc['2021-01-01':'2021-01-31'].to_numpy().flatten()

N = np.count_nonzero(Tide_Jan)

fft_output = rfft(Tide_Jan)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/1440)[:N//2] #Sample rate is 1440 points per day (24h/d*60min/h / 1 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.1)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.9)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o', color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.03), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='Sea level',fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\Sea_level_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\Sea_level_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

del Tide_Jan

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter']='Sea level (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = pd.DataFrame()
output_df = output_df.append(output, ignore_index=True)
output_df

## Baro pressure Jan 2021 fourier transform

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['baro_pressure_kPa']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col)

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylabel("Atmospheric pressure (kPa)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Baro_kPa_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
baro_pressure_kPa = df3['baro_pressure_kPa']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(baro_pressure_kPa)

fft_output = rfft(baro_pressure_kPa)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.04)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf)

ax.set_ylabel("Amplitude (kPa)")
ax.set_xlabel("Frequency (cpd)")
ax.set_ylim(0,0.4)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Baro_freq_202101_prominence004.png', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = 'Baro pressure (kPa)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## 3-month

In [ ]:
baro_pressure_kPa = df3['baro_pressure_kPa']['2021-01-01':'2021-03-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(baro_pressure_kPa)

fft_output = rfft(baro_pressure_kPa)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.04)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf)

ax.set_ylabel("Amplitude (kPa)")
ax.set_xlabel("Frequency (cpd)")
ax.set_ylim(0,1)
ax.set_xlim(0,5)
ax.plot(xf[peaks], yf[peaks], 'ro')

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Baro_freq_202101.png', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_path= os.path.relpath('../out/fourier_baro_20210101_20210331.csv')
output.to_csv(output_path)

## River hf Jan 2021 fourier transform

In [ ]:
river_hf_Lyt_m = df3['river_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['river_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylabel("Freshwater head (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\River_hf_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
np.count_nonzero(river_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(river_hf_Lyt_m)

fft_output = rfft(river_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.2)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.6)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='River freshwater head', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\River_hf_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\River_hf_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del river_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = 'River hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## River SC fourier transform

In [ ]:
river_cond_uScm = df3['river_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021,1,1)
datetime_end = datetime.date(2021,1,31)

x = df3.index

# GW SC
# East in ax1
y = df3['river_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylabel("River specific conductance (uS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

plt.style.use('default')

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\River_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
np.count_nonzero(river_cond_uScm)

In [ ]:
N = np.count_nonzero(river_cond_uScm)

fft_output = rfft(river_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=5000)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf,linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,15500)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+550), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='River SC', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\River_SC_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\River_SC_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del river_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter']='River SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['river_HOBO_temp']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylabel("Temperature ( C)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\River_HOBO_temp_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
river_HOBO_temp = df3['river_HOBO_temp']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(river_HOBO_temp)

fft_output = rfft(river_HOBO_temp)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.3)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf)

ax.set_ylabel("Amplitude (m)")
ax.set_xlabel("Frequency (cpd)")
ax.set_ylim(0,6)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'ro')

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.02), rotation=0)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\River_HOBO_temp_freq_202101.png', dpi=300, bbox_inches="tight")

plt.show()

## ParengaPl Jan 2021 fourier transform

### hf

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['ParengaPl_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(0,1)
ax.set_ylabel("Freshwater head (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\ParengaPl_hf_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
ParengaPl_hf_Lyt_m = df3['ParengaPl_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(ParengaPl_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(ParengaPl_hf_Lyt_m)

fft_output = rfft(ParengaPl_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.1)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.35)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.012), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='2-R freshwater head', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\ParengaPl_hf_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\ParengaPl_hf_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del ParengaPl_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '2-W ParengaPl hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

### SC

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['ParengaPl_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(16400,17500)
ax.set_ylabel(u"2-W spec. cond. (\u03bcS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\2-W_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
ParengaPl_cond_uScm = df3['ParengaPl_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(ParengaPl_cond_uScm)

In [ ]:
N = np.count_nonzero(ParengaPl_cond_uScm)

fft_output = rfft(ParengaPl_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=100)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)

ax.set_ylim(0,265)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+10), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='2-R SC', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\ParengaPl_SC_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\ParengaPl_SC_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del ParengaPl_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '2-W ParengaPl SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## APP192 Fourier transform

### hf

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['APP192_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(0.4,1)
ax.set_ylabel("Freshwater head (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\APP192_hf_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
APP192_hf_Lyt_m = df3['APP192_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(APP192_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(APP192_hf_Lyt_m)

fft_output = rfft(APP192_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.04)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.15)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.006), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='1-L2 freshwater head', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E2_hf_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E2_hf_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del APP192_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-E2 APP192 hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

### SC

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['APP192_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(2000,2600)
ax.set_ylabel(u"1-E2 spec. cond. (\u03bcS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-E2_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
APP192_cond_uScm = df3['APP192_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(APP192_cond_uScm)

fft_output = rfft(APP192_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=50)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,75)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+3), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

ax.set_title(label='1-L2 SC', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E2_SC_freq_202101.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E2_SC_freq_202101.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del APP192_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-E2 APP192 SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## OwlesTce fourier transform

### hf

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['OwlesTce_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(0.2,0.9)
ax.set_ylabel("Freshwater head (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-E1_hf_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
OwlesTce_hf_Lyt_m = df3['OwlesTce_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(OwlesTce_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(OwlesTce_hf_Lyt_m)

fft_output = rfft(OwlesTce_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.1)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.21)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color='#c34c54',markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.008), rotation=0,fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='1-L1 freshwater head', fontsize=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E1_hf_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E1_hf_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del OwlesTce_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-E1 OwlesTce hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

### SC

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['OwlesTce_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(1550,2000)
ax.set_ylabel(u"1-E1 spec. cond. (\u03bcS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-E1_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
OwlesTce_cond_uScm = df3['OwlesTce_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(OwlesTce_cond_uScm)

fft_output = rfft(OwlesTce_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=20)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,50)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+2), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='1-L1 SC', fontsize=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E1_SC_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-E1_SC_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del OwlesTce_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-E1 OwlesTce SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## 1-W MarigoldLn fourier transform

In [ ]:
MarigoldLn_hf_Lyt_m = df3['MarigoldLn_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(MarigoldLn_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(MarigoldLn_hf_Lyt_m)

fft_output = rfft(MarigoldLn_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.15)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.2)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color='#c34c54', markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.007), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='1-R freshwater head', fontsize=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-W_hf_freq_202101_prominence001_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-W_hf_freq_202101_prominence001_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del MarigoldLn_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-W MarigoldLn hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

### SC

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['MarigoldLn_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
#ax.set_ylim(1550,2000)
ax.set_ylabel(u"1-W spec. cond. (\u03bcS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-E1_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
MarigoldLn_cond_uScm = df3['MarigoldLn_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(MarigoldLn_cond_uScm)

fft_output = rfft(MarigoldLn_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=30)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,50)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+2), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='1-R SC', size=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-W_SC_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\1-W_SC_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del MarigoldLn_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '1-W MarigoldLn SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

## 2-E EvansAve fourier transform

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['EvansAve_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(0.2,1.2)
ax.set_ylabel("Freshwater head (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\2-E_hf_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
EvansAve_hf_Lyt_m = df3['EvansAve_hf_Lyt_m']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
np.count_nonzero(EvansAve_hf_Lyt_m)

In [ ]:
N = np.count_nonzero(EvansAve_hf_Lyt_m)

fft_output = rfft(EvansAve_hf_Lyt_m)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.2)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel("Amplitude (m)", fontsize=18)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,0.3)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color='#c34c54',markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.011), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='2-L freshwater head', fontsize=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\2-E_hf_freq_202101_prominence001_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\2-E_hf_freq_202101_prominence001_with_title.svg', dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
del EvansAve_hf_Lyt_m

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '2-E EvansAve hf (m)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

### SC

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 1, 31)

x = df3.index

# GW SC
# East in ax1
y = df3['EvansAve_cond_uScm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
ax.set_ylim(450,550)
ax.set_ylabel(u"2-E spec. cond. (\u03bcS/cm)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\2-E_SC_202101.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
EvansAve_cond_uScm = df3['EvansAve_cond_uScm']['2021-01-01':'2021-01-31'].to_numpy().flatten()

In [ ]:
N = np.count_nonzero(EvansAve_cond_uScm)

fft_output = rfft(EvansAve_cond_uScm)

yf = 2.0/N * np.abs(fft_output[0:N//2])

xf = rfftfreq(N, 1/144)[:N//2] #Sample rate is 144 points per day (24h/d*60min/h / 10 min)
# # Iterates until Nyquist frequency of N/2

peaks=sig.find_peaks(yf[xf >=0],prominence=0.8)[0]

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(8)
fig.set_figheight(5)

p1, = ax.plot(xf, yf, linewidth=2)

ax.set_ylabel(u"Amplitude (\u03bcScm$^{-1}$)", fontsize=17)
ax.set_xlabel("Frequency (cycles day$^{-1}$)", fontsize=17)
ax.set_ylim(0,4)
ax.set_xlim(0,4)
ax.plot(xf[peaks], yf[peaks], 'o',color=dot_colour, markersize=10)

for i, txt in enumerate(xf[peaks]):
    plt.annotate('{:.3f}'.format(txt), (xf[peaks][i], yf[peaks][i]+0.12), rotation=0, fontsize=16)

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

for axis in ["bottom","left"]:
    ax.spines[axis].set_linewidth(1)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.set_title(label='2-L SC', fontsize=18)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\2-E_SC_freq_202101_with_title.png', dpi=300, bbox_inches="tight")
#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Fourier_analysis\Increased_fontsize\2-E_SC_freq_202101_with_title.svg', dpi=300, bbox_inches="tight")

# NOTE the very small prominence

plt.show()

In [ ]:
del EvansAve_cond_uScm

In [ ]:
output = pd.DataFrame()
output['index'] = peaks
output['freq (cpd)'] = xf[peaks]
output['amplitude'] = np.abs(yf)[peaks]
output['Parameter'] = '2-E EvansAve SC (uS/cm)'
output = output.sort_values('amplitude', ascending=False)
output

In [ ]:
output_df = output_df.append(output, ignore_index=True)
output_df

In [ ]:
output_df.to_csv(os.path.relpath('../out/output_df_Jan2021.csv'))

## Cross-correlation

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

Baro = df3.loc[:,'baro_pressure_kPa'].values

for col in df3.columns:
    if "_hf_Lyt_m" in col:
        interesting_df[col] = df3[col]

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Baro_diff = Baro - Baro.mean()
    ccov = np.correlate(item_diff, Baro_diff, mode = 'full')
    ccor = ccov / (N * item.std() * Baro.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Baro', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor_hf_baro.csv')
df_ccor.to_csv(df_ccor_path)

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

Sea_level = Tide2.loc[:,'sea_level_m_Lyt'].values

for col in df3.columns:
    if "_hf_Lyt_m" in col:
        interesting_df[col] = df3[col]

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Sea_level_diff = Sea_level - Sea_level.mean()
    ccov = np.correlate(item_diff, Sea_level_diff, mode = 'full')
    ccor = ccov / (N * item.std() * Sea_level.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Sea_level', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

# Does 13 hr lag make sense?
# No! Should be around 2-3 hours max.
# Will slicing the data make a difference?

In [ ]:
Jan_df=df3.loc['2021-01-01':'2021-01-31']
Feb_df=df3.loc['2021-02-01':'2021-02-28']
Mar_df=df3.loc['2021-03-01':'2021-03-31']
Apr_df=df3.loc['2021-04-01':'2021-04-30']
May_df=df3.loc['2021-05-01':'2021-05-31']

In [ ]:
df6=df3.resample('1T').asfreq()

# Interpolate NaN values using third order spline function
df6 = df6.interpolate(method='polynomial', order=5)

In [ ]:
# Tide is every minute
Tide.head()

In [ ]:
Tide=Tide.resample('1T').asfreq()

In [ ]:
Jan_df=df6.loc['2021-01-01':'2021-01-02']
Jan_sea_level=Tide.loc['2021-01-01':'2021-01-02']

In [ ]:
# Transect 1 plot

#datetime_start = datetime.date(2021, 2, 1)
#datetime_end = datetime.date(2021, 2, 28)

x = Feb_df.index

y = Feb_df['river_hf_Lyt_m']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
#ax.set_xlim([datetime_start, datetime_end])
#ax.set_ylim(0,1)
ax.set_ylabel("Sea level (m LVD)")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Sea_level_mLVD_202102.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
# Slice df
#Feb_df=df3.loc['2021-02-01':'2021-02-09']

# Resample every min
#Feb_df=Feb_df.resample('1T').asfreq()

# Interpolate NaN values using 5th order polynomial function
#Feb_df = Feb_df.interpolate(method='polynomial', order=5)

#Slice sea level data
Feb_sea_level=Tide.loc['2021-02-01':'2021-02-28']

#Feb_sea_level=Feb_sea_level.resample('10T').asfreq()


col_dict = {}

df_ccor = pd.DataFrame()

interesting_df = pd.DataFrame()

Feb_sea_level_np = Feb_sea_level.loc[:,'sea_level_m_Lyt'].values

for col in Feb_df.columns:
    if "river_hf_Lyt_m" in col:
        interesting_df[col] = Feb_df[col]

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(Feb_sea_level_np)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Feb_sea_level_diff = Feb_sea_level_np - Feb_sea_level_np.mean()
    ccov = np.correlate(item_diff, Feb_sea_level_diff, mode = 'full')
    ccor = ccov / (N1 * item.std() * Feb_sea_level_np.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Sea_level', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N1':N1, 'N2':N2}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

In [ ]:
Feb_df['river_hf_Lyt_m'].count()

In [ ]:
Tide.loc['2021-02-01':'2021-02-28'].count()

In [ ]:
28*24*60

In [ ]:
#Select Var2: River
river_hf_Lyt_m_Feb = Feb_df.loc[:,'river_hf_Lyt_m'].values

# Feb 2021

interesting_df = pd.DataFrame()

# Select Var1: GW cond, exclude river cond
for col in Feb_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Feb)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Feb = river_hf_Lyt_m_Feb - river_hf_Lyt_m_Feb.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Feb, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Feb
df_ccor

In [ ]:
col_dict = {}

df_ccor = pd.DataFrame()

interesting_df = pd.DataFrame()

Jan_sea_level_np = Jan_sea_level.loc[:,'sea_level_m_Lyt'].values

for col in Jan_df.columns:
    if "_hf_Lyt_m" in col:
        interesting_df[col] = Jan_df[col]

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(Jan_sea_level_np)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Jan_sea_level_diff = Jan_sea_level_np - Jan_sea_level_np.mean()
    ccov = np.correlate(item_diff, Jan_sea_level_diff, mode = 'full')
    ccor = ccov / (N1 * item.std() * Jan_sea_level_np.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Sea_level', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N':N1}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor_GW_river_hf_sea_level.csv')
df_ccor.to_csv(df_ccor_path)

In [ ]:
# I think that exceeding a certain number of data points resulted in mistaken which variable comes first?

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

Sea_level = Tide2.loc[:,'sea_level_m_Lyt'].values

for col in df3.columns:
    if "_hf_Lyt_m" in col:
        interesting_df[col] = df3[col]

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Sea_level_diff = Sea_level - Sea_level.mean()
    ccov = np.correlate(item_diff, Sea_level_diff, mode = 'full')
    ccor = ccov / (N * item.std() * Sea_level.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Sea_level', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

# Does 13 hr lag make sense?
# No! Should be around 2-3 hours max.

## Bookmark

In [ ]:
col_dict = {}

Var1_df = pd.DataFrame()
df_ccor = pd.DataFrame()

Var2 = df3.loc[:, "river_hf_Lyt_m"].values

for col in df3.columns:
    if "_hf_Lyt_m" in col and "river" not in col:
        Var1_df[col] = df3[col]

for col in Var1_df.columns:
    Var1 = Var1_df.loc[:, {col}].values
    N = np.count_nonzero(Var1)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    Var1_diff= Var1 - Var1.mean()
    Var1_diff = Var1_diff.flatten()
    Var2_diff = Var2 - Var2.mean()
    ccov = np.correlate(Var1_diff, Var2_diff, mode = 'full')
    ccor = ccov / (N * Var1.std() * Var2.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Var2', 'date_time_start': Var1_df.index.min(), 'date_time_end': Var1_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del Var1_df
df_ccor

### monthly filters from df3

In [ ]:
import pandas as pd

In [ ]:
import datetime

In [ ]:
# GW SC and river hf

# Check if N Var1 = N var2 !

col_dict = {}

# Create empty dataframes for the variables
Var1_df = pd.DataFrame()
Var2_df = pd.DataFrame()

# Create empty dataframe for the results
df_ccor = pd.DataFrame()

# Select Var1: GW cond, exclude river cond. Append in Var1_df
for col in df3.columns:
    if "_cond_uScm" in col and 'river' not in col: # List Variable 1 here
        Var1_df[col] = df3[col]

# Select Var2: River hf
for col in df3.columns:
    if "river_hf_Lyt_m" in col: # Define Variable 2 here
        Var2_df[col]=df3[col]
        
for col in Var1_df.columns:
    Var1 = Var1_df.loc[:, {col}].values
    Var2 = Var2_df.loc[:, {col}].values
    N1 = np.count_nonzero(Var1)
    N2 = np.count_nonzero(Var2)
    if N1 == N2:
        N_check = "N1=N2"
    elif N1 < N2:
        N_check = "N1<N2"
    elif N1 > N2:
        N_check="N1>N2"
    else:
        raise Exception
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    Var1_diff= Var1 - Var1.mean()
    Var1_diff = Var1_diff.flatten()
    Var2 = Var2 - Var2.mean()
    ccov = np.correlate(Var1_diff, Var2, mode = 'full')
    ccor = ccov / (N1 * Var1.std() * Var2.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':Var2_df.columns[0], 'date_time_start': Var1.index.min(), 'date_time_end': Var1.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del Var1
del Var2

#Select Var2: River
river_hf_Lyt_m_Feb = Feb_df.loc[:,'river_hf_Lyt_m'].values

# Feb 2021

interesting_df = pd.DataFrame()

# Select Var1: GW cond, exclude river cond
for col in Feb_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Feb)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Feb = river_hf_Lyt_m_Feb - river_hf_Lyt_m_Feb.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Feb, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Feb

# Mar 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Mar = Mar_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in Mar_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Mar_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Mar)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Mar = river_hf_Lyt_m_Mar - river_hf_Lyt_m_Mar.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Mar, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Mar.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Mar

# Apr 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Apr = Apr_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in Apr_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Apr_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Apr)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Apr = river_hf_Lyt_m_Apr - river_hf_Lyt_m_Apr.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Apr, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Apr.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Apr

# May 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_May = May_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in May_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = May_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_May)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_May = river_hf_Lyt_m_May - river_hf_Lyt_m_May.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_May, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_May.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_May


df_ccor


In [ ]:
del Jan_df
del Feb_df
del Mar_df
del Apr_df
del May_df

### River cond vs. GW cond: Monthly

In [ ]:
# Check if N Var1 = N var2 !

# Create empty dictionary
col_dict = {}

# Create empty dataframe to select the interesting columns to be run as Variable 1. This will be deleted later.
interesting_df = pd.DataFrame()

# Create empty dataframe where the cross-correlation results will be appended in.
df_ccor = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Jan = Jan_df.loc[:,'river_cond_uScm'].values

# Jan 2021
# Select Var1: GW cond, exclude river cond
for col in Jan_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Jan_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_Jan)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Jan = river_cond_uScm_Jan - river_cond_uScm_Jan.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Jan, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_Jan.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Jan

#Select Var2: River
river_cond_uScm_Feb = Feb_df.loc[:,'river_cond_uScm'].values

# Feb 2021

interesting_df = pd.DataFrame()

# Select Var1: GW cond, exclude river cond
for col in Feb_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_Feb)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Feb = river_cond_uScm_Feb - river_cond_uScm_Feb.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Feb, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Feb

# Mar 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Mar = Mar_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in Mar_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Mar_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_Mar)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Mar = river_cond_uScm_Mar - river_cond_uScm_Mar.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Mar, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_Mar.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Mar

# Apr 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Apr = Apr_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in Apr_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Apr_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_Apr)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Apr = river_cond_uScm_Apr - river_cond_uScm_Apr.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Apr, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_Apr.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Apr

# May 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_May = May_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in May_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = May_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_May)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_May = river_cond_uScm_May - river_cond_uScm_May.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_May, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_May.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_May


df_ccor

In [ ]:
# Create empty dictionary
col_dict = {}

# Create empty dataframe to select the interesting columns to be run as Variable 1. This will be deleted later.
interesting_df = pd.DataFrame()

# Create empty dataframe where the cross-correlation results will be appended in.
df_ccor = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Jan = Jan_df.loc[:,'river_cond_uScm'].values

# Jan 2021
# Select Var1: GW cond, exclude river cond
for col in df3.columns:
    if "_cond_uScm" in col:
        interesting_df[col] = Jan_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_cond_uScm_Jan)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Jan = river_cond_uScm_Jan - river_cond_uScm_Jan.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Jan, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_cond_uScm_Jan.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Jan

df_ccor

In [ ]:
Jan_df['river_cond_uScm'].count()

In [ ]:
31*24*60/10

In [ ]:
# GW SC and river hf

df3=df3.resample('1T').asfreq()

# Interpolate NaN values using third order spline function
df3 = df3.interpolate(method='polynomial', order=5)

# Check if N Var1 = N var2 !

Jan_df=df3.loc['2021-01-01':'2021-01-31']
Feb_df=df3.loc['2021-02-01':'2021-02-28']
Mar_df=df3.loc['2021-03-01':'2021-03-31']
Apr_df=df3.loc['2021-04-01':'2021-04-30']
May_df=df3.loc['2021-05-01':'2021-05-31']

col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Jan = Jan_df.loc[:,'river_hf_Lyt_m'].values

# Jan 2021
# Select Var1: GW cond, exclude river cond
for col in Jan_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = Jan_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Jan)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Jan = river_hf_Lyt_m_Jan - river_hf_Lyt_m_Jan.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Jan, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Jan.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Jan

#Select Var2: River
river_hf_Lyt_m_Feb = Feb_df.loc[:,'river_hf_Lyt_m'].values

# Feb 2021

interesting_df = pd.DataFrame()

# Select Var1: GW cond, exclude river cond
for col in Feb_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Feb)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Feb = river_hf_Lyt_m_Feb - river_hf_Lyt_m_Feb.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Feb, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Feb

# Mar 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Mar = Mar_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in Mar_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = Mar_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Mar)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Mar = river_hf_Lyt_m_Mar - river_hf_Lyt_m_Mar.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Mar, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Mar.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Mar

# Apr 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Apr = Apr_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in Apr_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = Apr_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_Apr)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_Apr = river_hf_Lyt_m_Apr - river_hf_Lyt_m_Apr.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_Apr, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_Apr.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Apr

# May 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_May = May_df.loc[:,'river_hf_Lyt_m'].values

# Select Var1: GW cond, exclude river cond
for col in May_df.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = May_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m_May)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m_May = river_hf_Lyt_m_May - river_hf_Lyt_m_May.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m_May, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m_May.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_May


df_ccor


In [ ]:
# GW SC and river hf

# Check if N Var1 = N var2 !

col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m = df3.loc[:,'river_hf_Lyt_m'].values

# Jan 2021
# Select Var1: GW cond, exclude river cond
for col in df3.columns:
    if "river_cond_uScm" in col:
        interesting_df[col] = df3[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N1 = np.count_nonzero(item)
    N2 = np.count_nonzero(river_hf_Lyt_m)
    if N1 == N2:
        N_check = "N1=N2"
    lags = np.arange(- N1 + 1, N1)
    x = np.linspace(0, N1, N1)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_Lyt_m = river_hf_Lyt_m - river_hf_Lyt_m.mean()
    ccov = np.correlate(item_diff, river_hf_Lyt_m, mode = 'full')
    ccor = ccov / (N1 * item.std() * river_hf_Lyt_m.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N1, 'N_check':N_check}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m

df_ccor

In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor_river_SC_river_hf.csv')
df_ccor.to_csv(df_ccor_path)

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Jan = Jan_df.loc[:,'river_cond_uScm'].values

# Jan 2021
# Select Var1: GW cond, exclude river cond
for col in Jan_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Jan_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Jan = river_cond_uScm_Jan - river_cond_uScm_Jan.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Jan, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm_Jan.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_cond_uScm_Jan

# Feb 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Feb = Feb_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in Feb_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Feb = river_cond_uScm_Feb - river_cond_uScm_Feb.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Feb, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

# Mar 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Mar = Mar_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in Mar_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Mar_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Mar = river_cond_uScm_Mar - river_cond_uScm_Mar.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Mar, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm_Mar.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

# Apr 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_Apr = Apr_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in Apr_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = Apr_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_Apr = river_cond_uScm_Apr - river_cond_uScm_Apr.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_Apr, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm_Apr.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

# May 2021

interesting_df = pd.DataFrame()

#Select Var2: River
river_cond_uScm_May = May_df.loc[:,'river_cond_uScm'].values

# Select Var1: GW cond, exclude river cond
for col in May_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = May_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_uScm_May = river_cond_uScm_May - river_cond_uScm_May.mean()
    ccov = np.correlate(item_diff, river_cond_uScm_May, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm_May.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)


df_ccor

In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor_GW_SC_river_SC_monthly.csv')
df_ccor.to_csv(df_ccor_path)

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

#Select Var2: River
river_hf_Lyt_m_Jan = Jan_df.loc[:,'river_hf_Lyt_m'].values

for col in Jan_df.columns:
    if "_hf_Lyt_m" in col and 'river' not in col:
        interesting_df[col] = Jan_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m_Jan - river_hf_Lyt_m_Jan.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m_Jan.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Jan
df_ccor

In [ ]:
interesting_df = pd.DataFrame()

river_hf_Lyt_m_Feb = Feb_df.loc[:,'river_hf_Lyt_m'].values

for col in Feb_df.columns:
    if "_hf_Lyt_m" in col and 'river' not in col:
        interesting_df[col] = Feb_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m_Feb - river_hf_Lyt_m_Feb.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m_Feb.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Feb
df_ccor

In [ ]:
interesting_df = pd.DataFrame()

river_hf_Lyt_m_Mar = Mar_df.loc[:,'river_hf_Lyt_m'].values

for col in Mar_df.columns:
    if "_hf_Lyt_m" in col and 'river' not in col:
        interesting_df[col] = Mar_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m_Mar - river_hf_Lyt_m_Mar.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m_Mar.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Mar
df_ccor

In [ ]:
interesting_df = pd.DataFrame()

river_hf_Lyt_m_Apr = Apr_df.loc[:,'river_hf_Lyt_m'].values

for col in Apr_df.columns:
    if "_hf_Lyt_m" in col and 'river' not in col:
        interesting_df[col] = Apr_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m_Apr - river_hf_Lyt_m_Apr.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m_Apr.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_Apr
df_ccor

In [ ]:
interesting_df = pd.DataFrame()

river_hf_Lyt_m_May = May_df.loc[:,'river_hf_Lyt_m'].values

for col in May_df.columns:
    if "_hf_Lyt_m" in col and 'river' not in col:
        interesting_df[col] = May_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m_May - river_hf_Lyt_m_May.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m_May.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
del river_hf_Lyt_m_May
df_ccor



In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor_monthly.csv')
df_ccor.to_csv(df_ccor_path)

### Investigate cross-correlation during storm event

### River hf and GW SC

In [ ]:
storm_df=df3.loc['2021-05-30':'2021-06-06']

In [ ]:
col_dict = {}

interesting_df = pd.DataFrame()
df_ccor = pd.DataFrame()

river_cond_uScm = storm_df.loc[:,'river_cond_uScm'].values

for col in storm_df.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = storm_df[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_diff = river_cond_uScm - river_cond_uScm.mean()
    ccov = np.correlate(item_diff, river_cond_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

In [ ]:
# Resample df3 Jan-Feb, Mar-May: Difference in lag time?

In [ ]:
df_ccor_path= os.path.relpath('../out/df_ccor.csv')
df_ccor.to_csv(df_ccor_path)

In [ ]:
interesting_df = pd.DataFrame()

for col in df3.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = df3[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_cond_diff = river_cond_uScm - river_cond_uScm.mean()
    ccov = np.correlate(item_diff, river_cond_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_cond_uScm.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_cond_uScm', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor


In [ ]:
interesting_df = pd.DataFrame()

for col in df3.columns:
    if "_cond_uScm" in col and 'river' not in col:
        interesting_df[col] = df3[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_hf_diff = river_hf_Lyt_m - river_hf_Lyt_m.mean()
    ccov = np.correlate(item_diff, river_hf_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_hf_Lyt_m.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'River_hf_Lyt_m', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor


In [ ]:
interesting_df = pd.DataFrame()
river_odyssey_temp = df3.loc[:,'river_odyssey_temp'].values

for col in df3.columns:
    if "_temp" in col and 'river' not in col:
        interesting_df[col] = df3[col]
        
for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    river_temp_diff = river_odyssey_temp - river_odyssey_temp.mean()
    ccov = np.correlate(item_diff, river_temp_diff, mode = 'full')
    ccor = ccov / (N * item.std() * river_odyssey_temp.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'river_odyssey_temp', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor


### Difference in HOBO & Odyssey river temperatures?

#### The Odyssey temperature (deeper) colder than the HOBO temperature (shallower), which makes sense; the deeper, the colder the water.

In [ ]:
df3[["river_HOBO_temp", "river_odyssey_temp"]].tail()

# Plotting

In [ ]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import seaborn as sns

%matplotlib inline
#sns.set()

## River level & sea level

In [ ]:
Tide = pd.read_csv('../Tide/all.csv', delimiter = ',', header=None)

# Drop columns
Tide = Tide.drop([0], axis=1)

# Calculation for sea level relative to Lyttelton datum (sensor 41)
Tide[2] = Tide[2] - 1.75

Tide = Tide.rename(columns={1:"date_time", 2:"sea_level_m_Lyt"})

# Make Python recognize the date column as a datetime object
Tide['date_time']=pd.to_datetime(Tide['date_time'], format = '%Y-%m-%d %H:%M')

# Set date_time as the index
Tide.set_index('date_time', inplace = True)

In [ ]:
Tide.info()

In [ ]:
Tide.describe()

In [ ]:
Tide=Tide.resample('10T').asfreq()

In [ ]:
Tide2=Tide.loc['2021-01-01':'2021-06-09']

# Amplitude analysis

In [ ]:
# Filter df
df4=df3.loc['2021-02-01 00:00:00':'2021-02-01 12:00:00']

# Get minimum value of a single column 'y'
minValue = df4['river_level_Lyt_m'].min()
print("minimum value of 'river_level_Lyt_m': " , minValue)

# Get minimum value of a single column 'y'
maxValue = df4['river_level_Lyt_m'].max()
print("maximum value of 'river_level_Lyt_m': " , maxValue)

#Get amplitude
amplitude = maxValue - minValue
print("amplitude of 'river_level_Lyt_m' on 1 Feb 2021: " , amplitude)

In [ ]:
Tide2=Tide.loc['2021-02-01 00:00:00':'2021-02-01 12:00:00']

In [ ]:
# Get minimum value of a single column 'y'
minValue_sea = Tide2['sea_level_m_Lyt'].min()
print("minimum value of 'sea_level_m_Lyt': " , minValue_sea)

# Get minimum value of a single column 'y'
maxValue_sea = Tide2['sea_level_m_Lyt'].max()
print("maximum value of 'sea_level_m_Lyt': " , maxValue_sea)

#Get amplitude
amplitude_sea = maxValue_sea - minValue_sea
print("amplitude of 'sea_level_m_Lyt' on 1 Feb 2021: " , amplitude_sea)

In [ ]:
# Ratio of river amplitude and sea amplitude
amplitude_ratio = amplitude/amplitude_sea
print("amplitude ratio of 'river_level_Lyt_m' and 'sea_level_m_Lyt' on 1 Feb 2021: " , amplitude_ratio)

In [ ]:
print (df3['ParengaPl_temp'].index.min())
print (df3['ParengaPl_temp'].index.max())

In [ ]:
print (df3['MarigoldLn_temp'].index.min())
print (df3['MarigoldLn_temp'].index.max())

In [ ]:
print (df3['EvansAve_temp'].index.min())
print (df3['EvansAve_temp'].index.max())

In [ ]:
print (df3['BHBUR27_temp'].index.min())
print (df3['BHBUR27_temp'].index.max())

In [ ]:
print (df3['BHBUR27_temp'])

In [ ]:
# Are there rows that have NaN?
df3['ParengaPl_temp'].isnull().values.any()

In [ ]:
# How many NaN in the column?
df3['ParengaPl_temp'].isnull().sum()

In [ ]:
# List the index of ParengaPl NaN rows
df3[df3['ParengaPl_temp'].isnull()].index.tolist()

In [ ]:
# List the index of ParengaPl NaN rows
df3[df3['EvansAve_temp'].isnull()].index.tolist()

In [ ]:
# Interpolate NaN values using third order spline function
df3['EvansAve_temp'] = df3['EvansAve_temp'].interpolate(method='polynomial', order=5)

In [ ]:
# List the index of ParengaPl NaN rows
df3[df3['EvansAve_temp'].isnull()].index.tolist()

In [ ]:
# df2["OwlesTce_dhf_m"] = df2["river_hf_Lyt_m"] - df2["OwlesTce_hf_Lyt_m"]

In [ ]:
# List the index of ParengaPl NaN rows
df3[df3['OwlesTce_dhf_m'].isnull()].index.tolist()

In [ ]:
# List the index of ParengaPl NaN rows
df3[df3['river_hf_Lyt_m'].isnull()].index.tolist()

## Plotting v2

### Transect 1

In [ ]:
# COLOR SCHEMES
River_SC_col = '#0072B2'
River_temp_top_col = '#117733'
River_temp_bottom_col = '#DDCC77'
River_hf_col = '#88CCEE'

APP192_SC_col = '#009E73'
APP192_hf_col = '#006ddb'

OwlesTce_SC_col = '#D55E00'
OwlesTce_hf_col = '#b66dff'

MarigoldLn_SC_col = '#CC79A7'
ParengaPl_SC_col = '#E69F00'
EvansAve_SC_col = '#F0E442'

#APP207_col = 

In [ ]:
datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 9)


# Define x and y
x = df3.index
y1 = df3['river_level_Lyt_m']
y2 = df3['river_cond_uScm']
x3 = Tide.index
y3 = Tide['sea_level_m_Lyt']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(3)

twin1 = ax.twinx()

p1, = ax.plot(x, y1, color=River_SC_col, label="River level")
p2, = ax.plot(x3, y3, color='#83d982', label="Sea level")
p3, = twin1.plot(x, y2, color='#D81B60', label="River SC")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])

# Format date in x axis
myFmt = mdates.DateFormatter('%d %b %y')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax.set_ylabel("Level (m LVD)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")

#ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors='black', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

ax.legend(handles=[p1, p2, p3], loc=1, ncol=3)


# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\River_sea_levels_SC_20210201_20210209_v3.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, x3, y3, datetime_start, datetime_end

In [ ]:
datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 9)


# Define x and y
x = df3.index
y1 = df3['river_level_Lyt_m']
y2 = df3['river_cond_uScm']
x3 = Tide.index
y3 = Tide['sea_level_m_Lyt']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(3)

twin1 = ax.twinx()

p1, = ax.plot(x, y1, color=River_SC_col, label="River level")
p2, = ax.plot(x3, y3, color='#83d982', label="Sea level")
p3, = twin1.plot(x, y2, color='#D81B60', label="River SC")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])

# Format date in x axis
myFmt = mdates.DateFormatter('%d %b %y')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax.set_ylabel("Level (m LVD)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")

#ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors='black', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

ax.legend(handles=[p1, p2, p3], loc=1, ncol=3)


# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\River_sea_levels_SC_20210201_20210209_v4.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, x3, y3, datetime_start, datetime_end

In [ ]:
df3['river_cond_uScm'].loc[datetime.datetime(2021,2,2,6):datetime.datetime(2021,2,2,12)].min()

In [ ]:
print ("Min SC on 2 Feb 2021: " +  str(df3['river_cond_uScm'].loc[datetime.datetime(2021,2,2,6):datetime.datetime(2021,2,2,12)].min()))
print ("Percentage SC compared to seawater : " +  str(df3['river_cond_uScm'].loc[datetime.datetime(2021,2,2,6):datetime.datetime(2021,2,2,12)].min()*100/55000))
print ("Max SC on 2 Feb 2021: " +  str(df3['river_cond_uScm'].loc[datetime.datetime(2021,2,2,6):datetime.datetime(2021,2,2,12)].max()))
print ("Percentage SC compared to seawater : " +  str(df3['river_cond_uScm'].loc[datetime.datetime(2021,2,2,6):datetime.datetime(2021,2,2,12)].max()*100/55000))

In [ ]:
df3['river_cond_uScm'].loc[datetime.datetime(2021,2,7,20):datetime.datetime(2021,2,8,12)].min()

In [ ]:
df3['river_cond_uScm'].loc[datetime.datetime(2021,2,7,20):datetime.datetime(2021,2,8,12)].max()

In [ ]:
Tide['sea_level_m_Lyt'].loc[datetime_start:datetime_end].count()

In [ ]:
y1.count()

In [ ]:
x[0]

## Transect 1

### Storm event

In [ ]:
# Read rain csv file
rain = pd.read_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\Rainfall\Hourly_rainfall_BromleyEws43967_20210101_0000_20210929_1500.csv', delimiter = ',')

# Combine the date and time columns in rain
rain.insert(2, 'date_time', rain['date'] + ' ' + rain['time'])

# Make Python recognize the date_time column as a datetime object in the three dataframes
rain['date_time']=pd.to_datetime(rain['date_time'], format = '%d/%m/%Y %H:%S')

# Remove date and time columns in APP165
rain = rain.drop(["date", "time"], axis=1)

# Set date_time as the index of APP165_1
rain.set_index('date_time', inplace = True)

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 1, 1)
datetime_end = datetime.date(2021, 6, 9)

x = rain.index

y = rain['rain_mm']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(13)
fig.set_figheight(3)

p1, = ax.plot(x, y, color=River_SC_col, label="River")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])
#ax.set_ylim(0,1)
ax.set_ylabel("Rain (mm))")

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

tkw = dict(size=4, width=1.5)

ax.tick_params(axis='y', **tkw)
ax.tick_params(axis='x', **tkw)

#plt.title('River freshwater head')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\Sea_level_mLVD_202102.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 28)
datetime_end = datetime.date(2021, 6, 9)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(495,515) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_storm_20210528_20210609.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 29)
datetime_end = datetime.date(2021, 6, 9)

x = df3_hourly.index

# GW SC
# East in ax1
y1 = df3_hourly['OwlesTce_cond_uScm']
y2 = df3_hourly['APP192_cond_uScm']

# West in twin1
y3 = df3_hourly['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3_hourly['river_cond_uScm']

# River & GW hf in ax2
y5 = df3_hourly['river_hf_Lyt_m']
y6 = df3_hourly['OwlesTce_hf_Lyt_m']
y7 = df3_hourly['APP192_hf_Lyt_m']
y8 = df3_hourly['MarigoldLn_hf_Lyt_m']

y9 = df3_hourly['rain_mm']

y10=df3_hourly['OwlesTce_q0']
y11=df3_hourly['APP192_q0']
y12=df3_hourly['MarigoldLn_q0']

y13=df3_hourly['OwlesTce_xT_m']
y14=df3_hourly['APP192_xT_m']
y15=df3_hourly['MarigoldLn_xT_m']

fig, (ax3, ax2, ax4, ax5, ax1) = plt.subplots(5)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

p9, = ax3.plot(x, y9, color = 'black', label="Rain")

p10, = ax4.plot(x, y10, color=OwlesTce_SC_col, label="1-L1")
p11, = ax4.plot(x, y11, color=APP192_SC_col, label="1-L2")
p12, = ax4.plot(x, y12, color=MarigoldLn_SC_col, label="1-R")

p13, = ax5.plot(x, y13, color=OwlesTce_SC_col, label="1-L1")
p14, = ax5.plot(x, y14, color=APP192_SC_col, label="1-L2")
p15, = ax5.plot(x, y15, color=MarigoldLn_SC_col, label="1-R")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(495,515) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left \ngroundwater SC \n(\u03bcS/cm)")
twin1.set_ylabel(u"True right \ngroundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

ax4.set_ylabel("Freshwater \ndischarge, \n$q_0$ ($m^2$/hr)")
ax5.set_ylabel("Wedge toe \nlocation, $x_T$ (m)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax2.set_ylabel('Freshwater head, \n$h_f$ (m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

ax4.set_xlim([datetime_start, datetime_end])

ax5.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax4.set_xticklabels([])
ax5.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_storm_20210528_20210609_xT_q0.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10, y11, y12, y13, y14, y15

# Storm event increased the connection between 1-L1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-R and 1-L2, but became amplified/clearer in 1-L1 after the storm event.


In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 1)
datetime_end = datetime.date(2021, 5, 10)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1800,2800) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_dry_20210501_20210510.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 10)
datetime_end = datetime.date(2021, 5, 20)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2500) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_20210510_20210520.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 3, 1)
datetime_end = datetime.date(2021, 3, 12)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,3000) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_20210301_20210312.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 3, 13)
datetime_end = datetime.date(2021, 3, 23)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,3000) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_20210313_20210323.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 9)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']
y6 = df3['OwlesTce_hf_Lyt_m']
y7 = df3['APP192_hf_Lyt_m']
y8 = df3['MarigoldLn_hf_Lyt_m']

y9 = rain['rain_mm']

#plt.plot(xfit, yfit, '-', color='gray')

#plt.fill_between(xfit, yfit - dyfit, yfit + dyfit,
#                 color='gray', alpha=0.2)

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-E1")
p2, = ax1.plot(x, y2, color=APP192_SC_col, label="1-E2")
p3, = twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p4, = twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
p5, = ax2.plot(x, y5, color=River_SC_col, label="River")
p6, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
p7, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
p8, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

p9, = ax3.plot(x2, y9, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,3000) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p4.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p4.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y5, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-E1")
z3, = ax2.plot(x, y7, color=APP192_SC_col, label="1-E2")
z4, = ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3, z4], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T1_2plots_20210201_20210209_dry.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, y8, y9, x2

# Storm event increased the connection between 1-E1 and the river, shown by the increase in GW salinity and head amplitude?
# The tidal signal became less clear in 1-W and 1-E2, but became amplified/clearer in 1-E1 after the storm event.

In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

fig, (ax2,ax3, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flow \nvelocity (m/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_xticklabels([])
ax3.set_ylim(-1.1,1.2)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_velocity_20220218.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['APP192_OwlesTce_qz']

fig, (ax2,ax3, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Vertical flux \n(m/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_xticklabels([])
#ax3.set_ylim(-1.1,1.2)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_vertical_flux_20220218.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

fig, (ax2,ax3, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flow \nvelocity (m/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_xticklabels([])
ax3.set_ylim(-1.2,1.2)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_velocity_20220224_1008.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

y9_upper = df3['OwlesTce_river_q_md'] + df3['OwlesTce_velocity_SD']
y9_lower = df3['OwlesTce_river_q_md'] + df3['OwlesTce_velocity_SD']

y10_upper = df3['APP192_river_q_md'] + df3['APP192_velocity_SD']
y10_lower = df3['APP192_river_q_md'] + df3['APP192_velocity_SD']

y11_upper = df3['MarigoldLn_river_q_md'] + df3['MarigoldLn_velocity_SD']
y11_lower = df3['MarigoldLn_river_q_md'] + df3['MarigoldLn_velocity_SD']

fig, (ax2,ax3, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax3.fill_between(x, y9_upper, y9_lower, color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower, color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flow \nvelocity (m/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_xticklabels([])
ax3.set_ylim(-1.2,9)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_velocity_20220309_1456.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

y9_upper = df3['OwlesTce_river_q_md'] + df3['OwlesTce_qx_SD']
y9_lower = df3['OwlesTce_river_q_md'] - df3['OwlesTce_qx_SD']

y10_upper = df3['APP192_river_q_md'] + df3['APP192_qx_SD']
y10_lower = df3['APP192_river_q_md'] - df3['APP192_qx_SD']

y11_upper = df3['MarigoldLn_river_q_md'] + df3['MarigoldLn_qx_SD']
y11_lower = df3['MarigoldLn_river_q_md'] - df3['MarigoldLn_qx_SD']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax3.fill_between(x, y9_upper, y9_lower, color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower, color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower, color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flux, $q_x$\n($\mathregular{m^{3}/m^{2}}$/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-1.5,1.4)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.9,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_flux_20220330_1109.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

y9_upper = df3['OwlesTce_river_q_md'] + df3['OwlesTce_qx_SD']
y9_lower = df3['OwlesTce_river_q_md'] - df3['OwlesTce_qx_SD']

y10_upper = df3['APP192_river_q_md'] + df3['APP192_qx_SD']
y10_lower = df3['APP192_river_q_md'] - df3['APP192_qx_SD']

y11_upper = df3['MarigoldLn_river_q_md'] + df3['MarigoldLn_qx_SD']
y11_lower = df3['MarigoldLn_river_q_md'] - df3['MarigoldLn_qx_SD']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flux, $q_x$\n($\mathregular{m^{3}m^{-2}day^{-1}}$)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-1.5,1.4)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.9,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_flux_20220428_1552.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$dh_f/dx$ (-)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.1,0.1)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_HydraulicGradient_20220907_1223.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$i$ (-)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.1,0.1)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_HydraulicGradient_20220912_0852.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


# Presentation figures

In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$i$ (-)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.1,0.1)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

#ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
#ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
#ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Presentation\T1_3plots_20210201_20210204_dry_errorshade_HydraulicGradient_20221014_1231.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Hydraulic gradient middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2) = plt.subplots(1)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(1.9)

#twin1 = ax1.twinx()
#twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
#twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
#ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
#ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
#twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
#twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

# Hydraulic gradient subplot
#ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
#ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
#ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

#ax3.set_xlabel("Date time")
#ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
#twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax2.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax2.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

#ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
#twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
#twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

#twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

#ax1.tick_params(axis='y', **tkw)
#ax1.tick_params(axis='x', **tkw)

#twin1.tick_params(axis='y', **tkw)

#twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

#datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
#datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

#ax3.fill_between(x, y9_upper, y9_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=OwlesTce_SC_col, alpha=0.2)
#ax3.fill_between(x, y10_upper, y10_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=APP192_SC_col, alpha=0.2)
#ax3.fill_between(x, y11_upper, y11_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
#ax2.set_xticklabels([])
#ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

#ax3.set_ylabel('Hydraulic gradient,\n$i$ (-)')
#ax3.set_xlim([datetime_start, datetime_end])
#ax3.set_ylim(-0.1,0.1)
#ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

#ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
#ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
#ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Presentation\T1_HeadPlot_20210201_20210204_20221014_1228.png', dpi=300, bbox_inches="tight")

plt.show()

#del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Hydraulic gradient middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax3) = plt.subplots(2)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(3)

#twin1 = ax1.twinx()
#twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
#twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
#ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
#ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
#twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
#twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

# Hydraulic gradient subplot
#ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
#ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
#ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

#ax3.set_xlabel("Date time")
#ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
#twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax2.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax2.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

#ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
#twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
#twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

#twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

#ax1.tick_params(axis='y', **tkw)
#ax1.tick_params(axis='x', **tkw)

#twin1.tick_params(axis='y', **tkw)

#twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

#datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
#datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

#ax3.fill_between(x, y9_upper, y9_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=OwlesTce_SC_col, alpha=0.2)
#ax3.fill_between(x, y10_upper, y10_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=APP192_SC_col, alpha=0.2)
#ax3.fill_between(x, y11_upper, y11_lower,
#                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
#                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
#ax2.set_xticklabels([])
#ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

#ax3.set_ylabel('Hydraulic gradient,\n$i$ (-)')
#ax3.set_xlim([datetime_start, datetime_end])
#ax3.set_ylim(-0.1,0.1)
#ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

#ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
#ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
#ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\Presentation\T1_HeadPlot_20210201_20210204_20221014_1228.png', dpi=300, bbox_inches="tight")

plt.show()

#del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 5, 28, 0, 0)
datetime_end = datetime.datetime(2021, 6, 9, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

y12 = df3['APP207_WL_Lyt_m']

# Velocity middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")
ax2.plot(x, y12, color='grey',label="APP207")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
#twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
#ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])
#ax2.set_ylim(0.2,0.3)

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$dh_f/dx$ (-)')
ax3.set_xlim([datetime_start, datetime_end])
#ax3.set_ylim(-0.1,0.1)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line


# a, b, c labels
#ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
#ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
#ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_APP207_20210528_20210609_storm_HydraulicGradient_20220908_1114.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

y12 = df3['APP207_WL_Lyt_m']

# Velocity middle subplot
y9= df3['OwlesTce_dhf_dx']
y10= df3['APP192_dhf_dx']
y11= df3['MarigoldLn_dhf_dx']

y9_upper = df3['OwlesTce_dhf_dx'] + df3['OwlesTce_oi']
y9_lower = df3['OwlesTce_dhf_dx'] - df3['OwlesTce_oi']

y10_upper = df3['APP192_dhf_dx'] + df3['APP192_oi']
y10_lower = df3['APP192_dhf_dx'] - df3['APP192_oi']

y11_upper = df3['MarigoldLn_dhf_dx'] + df3['MarigoldLn_oi']
y11_lower = df3['MarigoldLn_dhf_dx'] - df3['MarigoldLn_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")
ax2.plot(x, y12, color='grey',label="APP207")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])
#ax2.set_ylim(0.2,0.3)

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$dh_f/dx$ (-)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.1,0.1)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line


# a, b, c labels
ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.065,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_APP207_20210201_20210204_HydraulicGradient_20220908_1116.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


## Transect 2

In [ ]:
# Transect 2 plot

x = df3.index

# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']
y5 = df3['EvansAve_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']

fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.24))

# GW SC plot
p1, = ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
p2, = twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
p4, = ax2.plot(x, y4, color=River_SC_col, label="River")
p5, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
p6, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])
ax1.set_ylim(495,515) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', **tkw)
twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')
ax2.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)]) # Could define date in another variable?

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T2_2plots_20210201_20210203.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6



### Storm event

In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 28)
datetime_end = datetime.date(2021, 6, 9)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']
y5 = df3['EvansAve_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']

y7 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
p2, = twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
p4, = ax2.plot(x, y4, color=River_SC_col, label="River")
p5, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
p6, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

p7, = ax3.plot(x2, y7, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(495,515) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p3.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T2_2plots_storm_20210528_20210609.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, x2



In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 5, 1)
datetime_end = datetime.date(2021, 5, 10)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']
y5 = df3['EvansAve_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']

y7 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
p2, = twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
p4, = ax2.plot(x, y4, color=River_SC_col, label="River")
p5, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
p6, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

p7, = ax3.plot(x2, y7, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(490,530) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p3.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T2_2plots_dry_20210501_20210510.png', dpi=300, bbox_inches="tight")

plt.style.use('default')

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, x2



In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 6, 10)
datetime_end = datetime.date(2021, 6, 20)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']
y5 = df3['EvansAve_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']

y7 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
p2, = twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
p4, = ax2.plot(x, y4, color=River_SC_col, label="River")
p5, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
p6, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

p7, = ax3.plot(x2, y7, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
#ax1.set_ylim(495,515) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p3.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T2_2plots_storm_20210528_20210609.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, x2



In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 9)

x = df3.index

x2 = rain.index

# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']
y5 = df3['EvansAve_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']

y7 = rain['rain_mm']

fig, (ax3, ax2, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(8)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
p1, = ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
p2, = twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
p4, = ax2.plot(x, y4, color=River_SC_col, label="River")
p5, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
p6, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

p7, = ax3.plot(x2, y7, color = 'black', label="Rain")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twin1)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(p3.get_color())

ax3.set_ylabel("Rainfall (mm/hour)")

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)

ax3.tick_params(axis='y', **tkw)
ax3.tick_params(axis='x', **tkw)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end]) # Could define date in another variable?

ax3.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])
ax3.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\T2_2plots_20210201_20210209_dry.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y7, x2



In [ ]:
# Transect 1 plot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-E")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-E")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")


ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
#twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"East groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"West groundwater SC (\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, hf\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(loc=1)

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_2plots_20210201_20210204_dry_errorshade.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower



In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 4)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']

# Velocity middle subplot
y7= df3['EvansAve_river_q_md']
y8= df3['ParengaPl_river_q_md']

fig, (ax2,ax3, ax1) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-L")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-R")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-L")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-R")

ax3.plot(x, y7, color=EvansAve_SC_col, label="2-L")
ax3.plot(x, y8, color=ParengaPl_SC_col, label="2-R")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax3
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flow \nvelocity (m/day)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_xticklabels([])
ax3.set_ylim(-0.4,0.4)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_3plots_20210201_20210204_dry_errorshade_velocity_20220218.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower



In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']

# Velocity middle subplot
y7= df3['EvansAve_river_q_md']
y8= df3['ParengaPl_river_q_md']

y7_upper = df3['EvansAve_river_q_md'] + df3['EvansAve_qx_SD']
y7_lower = df3['EvansAve_river_q_md'] - df3['EvansAve_qx_SD']

y8_upper = df3['ParengaPl_river_q_md'] + df3['ParengaPl_qx_SD']
y8_lower = df3['ParengaPl_river_q_md'] - df3['ParengaPl_qx_SD']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-L")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-R")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-L")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-R")

ax3.plot(x, y7, color=EvansAve_SC_col, label="2-L")
ax3.plot(x, y8, color=ParengaPl_SC_col, label="2-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcS/cm)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcS/cm)")
twin2.set_ylabel(u"River SC (\u03bcS/cm)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

ax3.fill_between(x, y7_upper, y7_lower, color=EvansAve_SC_col, alpha=0.2)
ax3.fill_between(x, y8_upper, y8_lower, color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax1
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flux, $q_x$\n($\mathregular{m^{3}/m^{2}}$/day)')

ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.35,0.45)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 532,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.32,'c',fontsize=16,fontweight="bold")


plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_3plots_20210201_20210204_dry_errorshade_flux_20220330_1713.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower, y7_upper, y7_lower, y8_upper, y8_lower



In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']

# Velocity middle subplot
y7= df3['EvansAve_river_q_md']
y8= df3['ParengaPl_river_q_md']

y7_upper = df3['EvansAve_river_q_md'] + df3['EvansAve_qx_SD']
y7_lower = df3['EvansAve_river_q_md'] - df3['EvansAve_qx_SD']

y8_upper = df3['ParengaPl_river_q_md'] + df3['ParengaPl_qx_SD']
y8_lower = df3['ParengaPl_river_q_md'] - df3['ParengaPl_qx_SD']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-L")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-R")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-L")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-R")

ax3.plot(x, y7, color=EvansAve_SC_col, label="2-L")
ax3.plot(x, y8, color=ParengaPl_SC_col, label="2-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC SC\n(\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 22, 40)

ax3.fill_between(x, y7_upper, y7_lower,
                  where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=EvansAve_SC_col, alpha=0.2)
ax3.fill_between(x, y8_upper, y8_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax1
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flux, $q_x$\n($\mathregular{m^{3}m^{-2}day^{-1}}$)')

ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.35,0.45)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 532,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.32,'c',fontsize=16,fontweight="bold")


plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_3plots_20210201_20210204_dry_errorshade_flux_20220428_1554.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower, y7_upper, y7_lower, y8_upper, y8_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end

In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']

# Velocity middle subplot
y7= df3['EvansAve_dhf_dx']
y8= df3['ParengaPl_dhf_dx']

y7_upper = df3['EvansAve_dhf_dx'] + df3['EvansAve_oi']
y7_lower = df3['EvansAve_dhf_dx'] - df3['EvansAve_oi']

y8_upper = df3['ParengaPl_dhf_dx'] + df3['ParengaPl_oi']
y8_lower = df3['ParengaPl_dhf_dx'] - df3['ParengaPl_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-L")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-R")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-L")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-R")

ax3.plot(x, y7, color=EvansAve_SC_col, label="2-L")
ax3.plot(x, y8, color=ParengaPl_SC_col, label="2-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC SC\n(\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 22, 40)

ax3.fill_between(x, y7_upper, y7_lower,
                  where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=EvansAve_SC_col, alpha=0.2)
ax3.fill_between(x, y8_upper, y8_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax1
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$dh_f/dx$ (-)')

ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.04,0.02)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 532,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.01,'c',fontsize=16,fontweight="bold")


plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_3plots_20210201_20210204_dry_errorshade_HydraulicGradient_20220907_1229.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower, y7_upper, y7_lower, y8_upper, y8_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 4, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['EvansAve_cond_uScm']

# West in twin1
y2 = df3['ParengaPl_cond_uScm']

# River SC in twin 2
y3 = df3['river_cond_uScm']

# River & GW hf in ax2
y4 = df3['river_hf_Lyt_m']

y4_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y4_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y5 = df3['EvansAve_hf_Lyt_m']

y5_upper = df3['EvansAve_hf_Lyt_m'] + df3['EvansAve_ohf']
y5_lower = df3['EvansAve_hf_Lyt_m'] - df3['EvansAve_ohf']

y6 = df3['ParengaPl_hf_Lyt_m']

y6_upper = df3['ParengaPl_hf_Lyt_m'] + df3['ParengaPl_ohf']
y6_lower = df3['ParengaPl_hf_Lyt_m'] - df3['ParengaPl_ohf']

# Velocity middle subplot
y7= df3['EvansAve_dhf_dx']
y8= df3['ParengaPl_dhf_dx']

y7_upper = df3['EvansAve_dhf_dx'] + df3['EvansAve_oi']
y7_lower = df3['EvansAve_dhf_dx'] - df3['EvansAve_oi']

y8_upper = df3['ParengaPl_dhf_dx'] + df3['ParengaPl_oi']
y8_lower = df3['ParengaPl_dhf_dx'] - df3['ParengaPl_oi']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=EvansAve_SC_col, label="2-L")
twin1.plot(x, y2, color=ParengaPl_SC_col, label="2-R")

# River SC plot
twin2.plot(x, y3, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y4, color=River_SC_col, label="River")
ax2.plot(x, y5, color=EvansAve_SC_col, label="2-L")
ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-R")

ax3.plot(x, y7, color=EvansAve_SC_col, label="2-L")
ax3.plot(x, y8, color=ParengaPl_SC_col, label="2-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(450,550) # y limits for east GW SC (ax1)
twin1.set_ylim(16350,17350) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC SC\n(\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y4_upper, y4_lower, alpha=0.2)
ax2.fill_between(x, y5_upper, y5_lower, color=EvansAve_SC_col, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=ParengaPl_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 22, 40)

ax3.fill_between(x, y7_upper, y7_lower,
                  where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=EvansAve_SC_col, alpha=0.2)
ax3.fill_between(x, y8_upper, y8_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=ParengaPl_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')

ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax2 and ax1
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Hydraulic gradient,\n$i$ (-)')

ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-0.02,0.04)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 532,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.03,'c',fontsize=16,fontweight="bold")


plt.style.use('default')

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T2_3plots_20210201_20210204_dry_errorshade_HydraulicGradient_20220912_0852.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6, y5_upper, y5_lower, y6_upper, y6_lower, y4_upper, y4_lower, y7_upper, y7_lower, y8_upper, y8_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


## West wells

In [ ]:
# 1-E plot
x = df3.index

    # GW SC
    
y1 = df3['MarigoldLn_cond_uScm']
y2 = df3['ParengaPl_cond_uScm']

    # River SC
y3 = df3['river_cond_uScm']

    # River & GW hf
y4 = df3['river_hf_Lyt_m']
y5 = df3['MarigoldLn_hf_Lyt_m']
y6 = df3['ParengaPl_hf_Lyt_m']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()

# GW SC plot
p1, = ax1.plot(x, y1, color=MarigoldLn_SC_col, label="1-W")
p2, = ax1.plot(x, y2, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p3, = twin1.plot(x, y3, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])
ax1.set_ylim(15000,17500)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"Groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")

twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

# ax1.legend(handles=[p1, p2, p3], loc=1)

z1, = ax2.plot(x, y4, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y5, color=MarigoldLn_SC_col, label="1-W")
z3, = ax2.plot(x, y6, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel("Freshwater head, hf (m LVD)")
ax2.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)]) # Could define date in another variable?

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2, z3], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

# plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\West_2plots_20210201_20210203.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4, y5, y6



In [ ]:
# 1-W plot
x = df3.index

    # GW SC
y1 = df3['MarigoldLn_cond_uScm']

    # River SC
y2 = df3['river_cond_uScm']

    # River & GW hf
y3 = df3['river_hf_Lyt_m']
y4 = df3['MarigoldLn_hf_Lyt_m']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()

# GW SC plot
p1, = ax1.plot(x, y1, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p2, = twin1.plot(x, y2, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])
ax1.set_ylim(15850,16000)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

ax1.set_ylabel(u"Groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")

twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

# ax1.legend(handles=[p1, p2, p3], loc=1)

z1, = ax2.plot(x, y3, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y4, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel("Freshwater head, hf (m LVD)")
ax2.set_xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)]) # Could define date in another variable?

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

# plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-W_2plots_20210201_20210203.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4



In [ ]:
# 1-W plot
x = df3.index

    # GW SC
y1 = df3['MarigoldLn_cond_uScm']

    # River SC
y2 = df3['river_cond_uScm']

    # River & GW hf
y3 = df3['river_hf_Lyt_m']
y4 = df3['MarigoldLn_hf_Lyt_m']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()

# GW SC plot
p1, = ax1.plot(x, y1, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p2, = twin1.plot(x, y2, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 1, 16)])
ax1.set_ylim(15850,15975)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

ax1.set_ylabel(u"Groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")
twin1.set_ylim(0,30000)

twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

# ax1.legend(handles=[p1, p2, p3], loc=1)

z1, = ax2.plot(x, y3, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y4, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel("Freshwater head, hf (m LVD)")
ax2.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 1, 16)]) # Could define date in another variable?
ax2.set_ylim(-0.3,1.1)

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

# plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-W_2plots_20210201.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4



In [ ]:
# 1-W plot
x = df3.index

    # GW SC
y1 = df3['MarigoldLn_cond_uScm']

    # River SC
y2 = df3['river_cond_uScm']

    # River & GW hf
y3 = df3['river_hf_Lyt_m']
y4 = df3['MarigoldLn_hf_Lyt_m']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()

# GW SC plot
p1, = ax1.plot(x, y1, color=MarigoldLn_SC_col, label="1-W")

# River SC plot
p2, = twin1.plot(x, y2, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 2, 0)])
ax1.set_ylim(15850,15975)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

ax1.set_ylabel(u"Groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")
twin1.set_ylim(0,30000)

twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

# ax1.legend(handles=[p1, p2, p3], loc=1)

z1, = ax2.plot(x, y3, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y4, color=MarigoldLn_SC_col, label="1-W")

ax2.set_ylabel("Freshwater head, hf (m LVD)")
ax2.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 2, 0)]) # Could define date in another variable?
ax2.set_ylim(-0.3,1.1)

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

# plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-W_2plots_20210201_20210202.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4



In [ ]:
# 1-W plot
x = df3.index

    # GW SC
y1 = df3['ParengaPl_cond_uScm']

    # River SC
y2 = df3['river_cond_uScm']

    # River & GW hf
y3 = df3['river_hf_Lyt_m']
y4 = df3['ParengaPl_hf_Lyt_m']


fig, (ax2, ax1) = plt.subplots(2)
fig.subplots_adjust(right=0.75)

twin1 = ax1.twinx()

# GW SC plot
p1, = ax1.plot(x, y1, color=ParengaPl_SC_col, label="2-W")

# River SC plot
p2, = twin1.plot(x, y2, color=River_SC_col, label="River")

ax1.set_xlabel("Date time")
ax1.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 2, 0)])
ax1.set_ylim(16500,17500)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax1.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

ax1.set_ylabel(u"Groundwater SC (\u03bcS/cm)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")
twin1.set_ylim(0,30000)

twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax1.tick_params(axis='y', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax1.tick_params(axis='x', **tkw)

# ax1.legend(handles=[p1, p2, p3], loc=1)

z1, = ax2.plot(x, y3, color=River_SC_col, label = "River")
z2, = ax2.plot(x, y4, color=ParengaPl_SC_col, label="2-W")

ax2.set_ylabel("Freshwater head, hf (m LVD)")
ax2.set_xlim([datetime.datetime(2021, 2, 1, 1), datetime.datetime(2021, 2, 2, 0)]) # Could define date in another variable?
ax2.set_ylim(-0.3,1.1)

# Remove x axis labels for ax2
ax2.set_xticklabels([])

# Legend ax2
ax2.legend(handles=[z1, z2], loc=1)

# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

# plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\1-W_2plots_20210201_20210202.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, y3, y4



In [ ]:
df_export = df3.loc[datetime.datetime(2021, 2, 1, 3, 0):datetime.datetime(2021, 2, 2, 0, 0), ['river_hf_Lyt_m','river_cond_uScm','OwlesTce_hf_Lyt_m','OwlesTce_cond_uScm','APP192_hf_Lyt_m','APP192_cond_uScm','MarigoldLn_hf_Lyt_m','MarigoldLn_cond_uScm','EvansAve_hf_Lyt_m','EvansAve_cond_uScm','ParengaPl_hf_Lyt_m','ParengaPl_cond_uScm']]
df_export.to_csv(os.path.relpath('../out/SequenceOfEvents_20210201.csv'))
del df_export

In [ ]:
# Filter df
df4=df3.loc['2021-02-01 00:00:00':'2021-02-01 12:00:00']

In [ ]:
calc_df = pd.DataFrame()

for col in df4.columns:
    if "_hf_Lyt_m" in col:
        calc_df[col] = df4[col]
        
for col in calc_df.columns:
    minValue = calc_df[{col}].min()
    maxValue = calc_df[{col}].max()
    amplitude = maxValue - minValue
    print(amplitude)

del calc_df

In [ ]:
calc_df = pd.DataFrame()

for col in df4.columns:
    if "_cond_uScm" in col:
        calc_df[col] = df4[col]
        
for col in calc_df.columns:
    minValue = calc_df[{col}].min()
    maxValue = calc_df[{col}].max()
    amplitude = maxValue - minValue
    print(amplitude)
    
del calc_df

In [ ]:


# Get minimum value of a single column 'y'
minValue = df4['river_level_Lyt_m'].min()
print("minimum value of 'river_level_Lyt_m': " , minValue)

# Get minimum value of a single column 'y'
maxValue = df4['river_level_Lyt_m'].max()
print("maximum value of 'river_level_Lyt_m': " , maxValue)

#Get amplitude
amplitude = maxValue - minValue
print("amplitude of 'river_level_Lyt_m' on 1 Feb 2021: " , amplitude)

In [ ]:
df_count = pd.DataFrame()

In [ ]:
countPositive = df3.APP192_dSC_uScm[df3.APP192_dSC_uScm > 0].count()
countNegative = df3.APP192_dSC_uScm[df3.APP192_dSC_uScm < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)

timeNegative = 100 - timePositive

if countPositive + countNegative == df3.APP192_dSC_uScm.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "APP192_dSC", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)


In [ ]:
countPositive = df3.ParengaPl_dSC_uScm[df3.ParengaPl_dSC_uScm > 0].count()
countNegative = df3.ParengaPl_dSC_uScm[df3.ParengaPl_dSC_uScm < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.ParengaPl_dSC_uScm.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "ParengaPl_dSC", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)


In [ ]:
countPositive = df3.MarigoldLn_dSC_uScm[df3.MarigoldLn_dSC_uScm > 0].count()
countNegative = df3.MarigoldLn_dSC_uScm[df3.MarigoldLn_dSC_uScm < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.MarigoldLn_dSC_uScm.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "MarigoldLn_dSC", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

df_count

In [ ]:
# Evans Ave

countPositive = df3.EvansAve_dSC_uScm[df3.EvansAve_dSC_uScm > 0].count()
countNegative = df3.EvansAve_dSC_uScm[df3.EvansAve_dSC_uScm < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.EvansAve_dSC_uScm.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "EvansAve_dSC", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

df_count

In [ ]:
# OwlesTce

countPositive = df3.OwlesTce_dSC_uScm[df3.OwlesTce_dSC_uScm > 0].count()
countNegative = df3.OwlesTce_dSC_uScm[df3.OwlesTce_dSC_uScm < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.OwlesTce_dSC_uScm.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "OwlesTce_dSC", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

df_count

In [ ]:
# APP192

countPositive = df3.APP192_dhf_m[df3.APP192_dhf_m > 0].count()
countNegative = df3.APP192_dhf_m[df3.APP192_dhf_m < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)

timeNegative = 100 - timePositive

if countPositive + countNegative == df3.APP192_dhf_m.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "APP192_dhf", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

# ParengaPl

countPositive = df3.ParengaPl_dhf_m[df3.ParengaPl_dhf_m > 0].count()
countNegative = df3.ParengaPl_dhf_m[df3.ParengaPl_dhf_m < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.ParengaPl_dhf_m.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "ParengaPl_dhf", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

# MarigoldLn

countPositive = df3.MarigoldLn_dhf_m[df3.MarigoldLn_dhf_m > 0].count()
countNegative = df3.MarigoldLn_dhf_m[df3.MarigoldLn_dhf_m < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.MarigoldLn_dhf_m.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "MarigoldLn_dhf", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

# Evans Ave

countPositive = df3.EvansAve_dhf_m[df3.EvansAve_dhf_m > 0].count()
countNegative = df3.EvansAve_dhf_m[df3.EvansAve_dhf_m < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.EvansAve_dhf_m.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "EvansAve_dhf", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

# OwlesTce

countPositive = df3.OwlesTce_dhf_m[df3.OwlesTce_dhf_m > 0].count()
countNegative = df3.OwlesTce_dhf_m[df3.OwlesTce_dhf_m < 0].count()

timePositive = countPositive * 100 / (countPositive + countNegative)
print("% time River SC > GW SC: " , timePositive)

timeNegative = 100 - timePositive
print("% time GW SC > River SC: " , timeNegative)

if countPositive + countNegative == df3.OwlesTce_dhf_m.count():
    N_check = "N=N"
else:
    N_check = "N unequal"

d = {'Var': "OwlesTce_dhf", 'date_time_start': df3.index.min(), 'date_time_end': df3.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative, 'N_check': N_check}
df_count = df_count.append(d, ignore_index=True)

df_count


In [ ]:
df_count_path= os.path.relpath('../out/df_count_20210101_20210609.csv')
df_count.to_csv(df_count_path)

In [ ]:
# CREATE code for % time

count_cols = pd.DataFrame()
df_count = pd.DataFrame()

for col in df3.columns:
    if "_dSC_uScm" in col:
        count_cols[col] = df3[col]

for col in count_cols.columns:
    print(count_cols[col])
    #if count_cols[col] > 0:
    #    countPositive = col.count()

In [ ]:
for col in count_cols.columns:
    
    if col > 0:
        countPositive = count_cols[count_cols.col > 1].sum()
        
    elif a == b:
        print("a and b are equal")
    
    countPositive = count_cols[count_cols > 0].count()
    countNegative = count_cols[count_cols < 0].count()
    timePositive = countPositive * 100 / (countPositive + countNegative)
    timeNegative = 100 - timePositive
    d = {'Var': col, 'date_time_start': count_cols.index.min(), 'date_time_end': count_cols.index.max(), 'countPositive': countPositive, 'countNegative': countNegative, 'time_RiverSC>GWSC_%': timePositive, 'time_GWSC>RiverSC_%': timeNegative}
    df_count = df_count.append(d, ignore_index=True)

del count_cols
df_count
#----

for col in interesting_df.columns:
    item = interesting_df.loc[:, {col}].values
    N = np.count_nonzero(item)
    lags = np.arange(- N + 1, N)
    x = np.linspace(0, N, N)
    item_diff= item - item.mean()
    item_diff = item_diff.flatten()
    Sea_level_diff = Sea_level - Sea_level.mean()
    ccov = np.correlate(item_diff, Sea_level_diff, mode = 'full')
    ccor = ccov / (N * item.std() * Sea_level.std())
    maxlag = lags[np.argmax(ccor)]
    d = {'Var1': col, 'Var2':'Sea_level', 'date_time_start': interesting_df.index.min(), 'date_time_end': interesting_df.index.max(), 'Max_ccor':ccor.max(), 'Lag_min': maxlag*10, 'Lag_hr': maxlag*10/60, 'N':N}
    df_ccor = df_ccor.append(d, ignore_index=True)

del interesting_df
df_ccor

# Does 13 hr lag make sense?
# No! Should be around 2-3 hours max.
# Will slicing the data make a difference?

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River SC (μS/cm)')
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-W (μS/cm)')
df3['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-W (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
df3['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
df5['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=3)

df5.river_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Avon River SC (μS/cm)')
plt.ylabel('River SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
df5['river_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River hf (Lyt m)')
plt.ylabel('River hf (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=3)

df5.river_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Avon River SC (μS/cm)')
plt.ylabel('River SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
rain.info()

In [ ]:
# Add rainfall data

df5['river_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River hf (Lyt m)')
plt.ylabel('River hf (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=3)

rain.rain_mm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Rainfall')
plt.ylabel('Rainfall (mm/hour)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
# Add rainfall data

df5['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River SC')
plt.ylabel('River SC (uS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=3)

rain.rain_mm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Rainfall')
plt.ylabel('Rainfall (mm/hour)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
df5['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
df5['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')

plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=2)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
df5['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=2)

df5.OwlesTce_hf_Lyt_m.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Hf at 1-E1 (m Lyt)')
plt.ylabel('Hf (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])


# cross correlation between GW SC and GW hf during storm; hypothesize greater than non-storm periods
# Why? try plotting delta hf vs SC at 1-E1 during storm
# King tide before the storm - does it have an impact?
# Try monthly cross-correlation between GW SC and river hf (and river SC too)

In [ ]:
for c in df5:
    print(c)

In [ ]:
# Delta hf vs SC at Owles Tce or 1-E1
df5['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=2)

df5.OwlesTce_dhf_m.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='dhf at 1-E1 (m Lyt)')
plt.ylabel('dhf (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
df5['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=2)

df5.APP192_hf_Lyt_m.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Hf at 1-E2 (m Lyt)')
plt.ylabel('Hf (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large',loc=3)

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])


# cross correlation between GW SC and GW hf during storm; hypothesize greater than non-storm periods
# Why? try plotting delta hf vs SC at 1-E1 during storm
# King tide before the storm - does it have an impact?

In [ ]:
df5=df3.resample('10T').asfreq()
df5

In [ ]:
df5['river_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River hf (m Lyt)')
df5['APP192_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater hf at 1-E2 (m Lyt)')
df5['OwlesTce_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater hf at 1-E1 (m Lyt)')
plt.ylabel('Groundwater hf (m Lyt)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
df5['river_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'River hf (m Lyt)')
df5['APP192_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater hf at 1-E2 (m Lyt)')
df5['OwlesTce_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater hf at 1-E1 (m Lyt)')
plt.ylabel('Groundwater hf (m Lyt)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 5, 17), datetime.date(2021, 6, 10)])

In [ ]:
df2['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
df2['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
df2['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

#### Descriptive statistics

In [ ]:
SC_stats=df3[["OwlesTce_cond_uScm", "APP192_cond_uScm", "MarigoldLn_cond_uScm", "EvansAve_cond_uScm", "ParengaPl_cond_uScm", "river_cond_uScm"]].describe()
SC_stats_path= os.path.relpath('../out/SC_stats_20210101_20210609.csv')
SC_stats.to_csv(SC_stats_path)
SC_stats

In [ ]:
del SC_stats

In [ ]:
# Checking the SC
df2['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
# Checking the SC
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
# Checking the SC
df3['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
# Checking the SC
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 15), datetime.date(2021, 1, 25)])

# After salinity calibration of logger!

In [ ]:
# Checking the SC
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.datetime(2021, 4, 16, 9), datetime.datetime(2021, 4, 16, 13)])

# Marigold Ln : 2021, 4, 16, 11, 0

In [ ]:
# Checking the SC
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 4, 16), datetime.date(2021, 4, 18)])

In [ ]:
# Checking the SC
df3['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-W (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
# Checking the SC
df3['EvansAve_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-E (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

# Calibration raised the baseline of the SC?!

In [ ]:
# Checking the SC
df3['EvansAve_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-E (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 17), datetime.date(2021, 1, 25)])

# Calibration raised the baseline of the SC?!

In [ ]:
# Checking the SC
df3['EvansAve_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-E (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')



In [ ]:
df2['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
df2['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(15000,17000)
plt.legend(fontsize = 'xx-large')

In [ ]:
df2['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-W (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
monthly_mean = df2.groupby(pd.PeriodIndex(df2.index, freq="M"))['APP192_cond_uScm', 'APP192_dhf_m', 'OwlesTce_cond_uScm', 'OwlesTce_dhf_m', 'MarigoldLn_cond_uScm', 'EvansAve_cond_uScm', 'ParengaPl_cond_uScm'].mean()
monthly_mean

In [ ]:
monthly_mean['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
monthly_mean['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')

plt.ylabel('Monthly mean groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
monthly_mean['OwlesTce_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Δhf River - 1-E1 (μS/cm)')
monthly_mean['APP192_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Δhf River - 1-E2 (μS/cm)')
plt.ylabel('Monthly mean Δhf River - groundwater (m)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
monthly_mean_all = df2.groupby(pd.PeriodIndex(df2.index, freq="M"))['APP192_cond_uScm', 'OwlesTce_cond_uScm', 'MarigoldLn_cond_uScm', 'EvansAve_cond_uScm', 'ParengaPl_cond_uScm'].mean()
monthly_mean_all

In [ ]:
monthly_mean_all['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E1 (μS/cm)')
monthly_mean_all['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-E2 (μS/cm)')
plt.ylabel('Monthly mean groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

In [ ]:
monthly_mean['EvansAve_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-E (μS/cm)')
plt.ylabel('Monthly mean groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')
plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
monthly_mean['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 1-W (μS/cm)')
monthly_mean['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-W (μS/cm)')

plt.ylabel('Monthly mean groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')
plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
monthly_mean_all['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at 2-W (μS/cm)')
plt.ylabel('Monthly mean groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

#### Trying to display the min & max dates of specific columns

In [ ]:
print (df3['OwlesTce_cond_uScm'].index.min())
print (df3['OwlesTce_cond_uScm'].index.max())

In [ ]:
print (df3['BHBUR27_cond_uScm'].index.min())
print (df3['BHBUR27_cond_uScm'].index.max())

In [ ]:
print (df3['BHBUR27_cond_uScm'].index.min(skipna=True))
print (df3['BHBUR27_cond_uScm'].index.max(skipna=True))

In [ ]:
#Maybe with loc function?

In [ ]:
av_column_df3 = df3.mean(axis=0)
print (av_column_df3)

### Investigating the extreme dips

#### Dips due to salinity profile (logger wasn't turned off). Dips also associated with download times.

##### OwlesTce 16 April 2021 14:10 - 14:20 readings to be deleted

In [ ]:
df3['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at East 2 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.datetime(2021, 4, 16, 13), datetime.datetime(2021, 4, 16, 15)])

# Successful deletion

##### APP192 readings 18 Jan 2021 13:20 should be deleted

In [ ]:
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at East 1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.datetime(2021, 1, 18, 12), datetime.datetime(2021, 1, 18, 14)])
# Successful deletion

##### ParengaPl reading 18 Jan 2021 11:20 to be deleted

In [ ]:
df3['ParengaPl_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at West 1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.datetime(2021, 1, 18, 9), datetime.datetime(2021, 1, 18, 12)])

# Successfully deleted

## Which changes first in GW: Level, SC, temp?

### APP192: Head vs. SC

##### SC peak lags behind head peak

In [ ]:
df3['APP192_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.legend(fontsize = 'xx-large')

df3.APP192_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 SC')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])

###### Closer look

In [ ]:
df3['APP192_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.ylim(0.4, 1.2)
plt.legend(fontsize = 'xx-large')

df3.APP192_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 SC')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(2000, 2600)
plt.xlim([datetime.datetime(2021, 2, 1, 2), datetime.datetime(2021, 2, 1, 18)])

### APP192: Head vs. temp

##### Temp peak lags behind head peak

In [ ]:
df3['APP192_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.legend(fontsize = 'xx-large')

df3.APP192_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Temperature ( C)', fontsize=20)
plt.ylim(15.5,16)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])

###### Closer look

###### Temp peak came later than SC peak

In [ ]:
df3['APP192_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.ylim(0.4, 1.2)
plt.legend(fontsize = 'xx-large')

df3.APP192_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater temperature ( C)', fontsize=20)
plt.ylim(15.6,16)
plt.xlim([datetime.datetime(2021, 2, 1, 2), datetime.datetime(2021, 2, 1, 22)])

### APP192: SC vs. temp

In [ ]:
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 SC')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large')

df3.APP192_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Temperature ( C)', fontsize=20)
plt.ylim(15.5,16)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 3)])

###### Closer look

In [ ]:
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'APP192 SC')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large')

df3.APP192_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='APP192 temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater temperature ( C)', fontsize=20)
plt.ylim(15.6,16)
plt.xlim([datetime.datetime(2021, 2, 1, 2), datetime.datetime(2021, 2, 1, 22)])

### So... Order of GW fluctuation as a result of tide (river head & salinity change) seems to be: Head, salinity, temp. What about other wells?

#### MarigoldLn: temp peaks first before salinity peaks? See below
#### Is this to do with the K value potentially?

### MarigoldLn: Head vs. SC

#### SC peaks lag behind head peaks

In [ ]:
df3['MarigoldLn_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'MarigoldLn hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.legend(fontsize = 'xx-large')
plt.ylim(-1,1)

df3.MarigoldLn_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='MarigoldLn SC')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(15800,16300)
plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4)])

##### Closer look: SC peaks lag behind head peaks by ~10h? Does this make sense?
##### GW head peak ~8:00, SC peak ~ 18:00

In [ ]:
df3['MarigoldLn_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'MarigoldLn hydraulic head')
plt.ylabel('Hydraulic head (m LVD)', fontsize=20)
plt.legend(fontsize = 'xx-large')
plt.ylim(-1,1)

df3.MarigoldLn_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='MarigoldLn SC')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(15800,16300)
plt.xlim([datetime.datetime(2021, 1, 1, 3), datetime.datetime(2021, 1, 1, 21)])

### MarigoldLn: Head vs. temp

#### Tiny ~0.01 °C change in temp with head fluctuation
#### ~6h lag between head peak and temp peak

In [ ]:
df3['MarigoldLn_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'MarigoldLn hydraulic head')
plt.ylabel('Hydraulic head(m LVD)', fontsize=20)
plt.ylim(-1,1)
plt.legend(fontsize = 'xx-large')

df3.MarigoldLn_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='MarigoldLn temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Temperature ( C)', fontsize=20)
plt.ylim(13.275,13.325)
plt.xlim([datetime.datetime(2021, 1, 1), datetime.datetime(2021, 1, 4)])

#### Indiscernible change in temp with head fluctuation?

In [ ]:
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'MarigoldLn SC')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(15570,16250)
plt.legend(fontsize = 'xx-large')

df3.MarigoldLn_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='MarigoldLn temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Temperature ( C)', fontsize=20)
plt.ylim(13.45,13.6)

plt.xlim([datetime.date(2021, 1, 25), datetime.date(2021, 2, 3)])

### MarigoldLn: SC vs. temp

#### Temp peaks preceed SC peaks by ~3h?

In [ ]:
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'MarigoldLn SC')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)
plt.ylim(16000,16200)
plt.legend(fontsize = 'xx-large')

df3.MarigoldLn_temp.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='MarigoldLn temp')
plt.legend(fontsize = 'xx-large', loc=3)
plt.ylabel('Temperature ( C)', fontsize=20)
plt.ylim(13.275,13.325)

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4)])

In [ ]:
df3['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at East 2 (μS/cm)')
df3['APP192_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label = 'Groundwater SC at East 1 (μS/cm)')
plt.ylabel('Groundwater SC (μS/cm)', fontsize=20)

plt.legend(fontsize = 'xx-large')

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 20)])

In [ ]:
df3['MarigoldLn_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Groundwater SC at West 1 (μS/cm)')

In [ ]:
df3['ParengaPl_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20)
df3['EvansAve_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.legend(fontsize = 'xx-large', loc=3)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 3, 1)])
plt.ylim(-0.75,0.75)

In [ ]:
df3.info()

## Δ SC vs. Δ H in Transect 2

### Δ SC: The closer you are to zero, the more similar GW SC with river SC.

I.e., ΔSC = 0 when River SC = GW SC

In [ ]:
df3['ParengaPl_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - West 2 (m)')
df3['EvansAve_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δh (m)', fontsize=20)
plt.ylim(-1.5,3.5)

df3.ParengaPl_delta_SC_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Δ SC River - West 2 (μS/cm)')
df3.EvansAve_delta_SC_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='Δ SC River - East 3 (μS/cm)')
plt.ylabel('ΔSC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])


In [ ]:
Extract = df3[['river_cond_uScm','ParengaPl_cond_uScm','ParengaPl_delta_SC_uScm','EvansAve_cond_uScm','EvansAve_delta_SC_uScm']]
Extract.loc['2021-02-01 10:00:00':'2021-02-01 13:00:00']

In [ ]:
df3['ParengaPl_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δhf River - 2-W (m)')
df3['EvansAve_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δhf River - 2-E (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δhf (m)', fontsize=20)
plt.ylim(-1,1)
plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 5, 15)])

In [ ]:
# Get minimum value of a single column 'y'
aveValue = River['river_cond_uScm'].min()
print("minimum value of 'river_cond_uScm': " , minValue)

In [ ]:
# Get minimum value of a single column 'y'
minValue = River['river_cond_uScm'].max()
print("maximum value of 'river_cond_uScm': " , minValue)

In [ ]:
table = pd.pivot_table(df3,index=['APP192_dhf_m','OwlesTce_dhf_m'],aggfunc={'APP192_dhf_m':np.mean,'OwlesTce_dhf_m':np.mean})
table

In [ ]:
df3['APP192_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δhf River - 1-E2 (m)')
df3['OwlesTce_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δhf River - 1-E1 (m)')
df3['MarigoldLn_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δhf River - 1-W (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δhf (m)', fontsize=20)

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 10)])

In [ ]:
df3['ParengaPl_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - West 2 (m)')
df3['EvansAve_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δh (m)', fontsize=20)
plt.ylim(-1.5,3.5)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='SC at East 3 or EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

In [ ]:
df3['ParengaPl_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - West 2 (m)')
df3['EvansAve_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δh (m)', fontsize=20)
plt.ylim(-1.5,3.5)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='SC at East 3 or EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])


#### Conversion to hf didn't change the hydraulic gradient or direction of flow much...

In [ ]:
df3['MarigoldLn_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - West 1 (m)')
df3['APP192_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 1 (m)')
df3['OwlesTce_dhf_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 2 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δh (m)', fontsize=20)
plt.ylim(-1.5,3.5)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

#### Did the conversion to hf change the flow direction from APP192 to OwlesTce? No.

In [ ]:
df3['MarigoldLn_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf West 1 or MarigoldLn (m)')
df3['APP192_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf East 1 or APP192 (m)')
df3['OwlesTce_hf_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf East 2 or Owles Tce (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('hf (m)', fontsize=20)
plt.ylim(-1.5,3.5)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

In [ ]:
df3['MarigoldLn_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf West 1 or MarigoldLn (m)')
df3['APP192_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf East 1 or APP192 (m)')
df3['OwlesTce_WL_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Hf East 2 or Owles Tce (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('WL (m)', fontsize=20)
plt.ylim(-1.5,3.5)
plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

In [ ]:
df3['MarigoldLn_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - West 1 (m)')
df3['APP192_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 1 (m)')
df3['OwlesTce_delta_h_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δh River - East 2 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δh (m)', fontsize=20)
plt.ylim(-1.5,3.5)

df3.MarigoldLn_delta_SC_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='Δ SC River - West 1 (μS/cm)')
df3.APP192_delta_SC_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='Δ SC River - East 1 (μS/cm)')
df3.OwlesTce_delta_SC_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='Δ SC River - East 2 (μS/cm)')
plt.ylabel('ΔSC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])


#### Does the temp difference between river and GW drive increase in GW SC (i.e. drive saltwater intrusion)?

#### E.g., summer temperature (river warmer than GW) and winter (river colder than GW)

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='SC at East 3 or EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 2, 4)])

#### Interesting trend here: As river becomes cooler than GW approaching winter, the GW becomes more saline. Are there other factors at play here?

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 10)])

#### Not much trend in river SC over the seasons, except freshening due to storm in late May

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.river_cond_mScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 1, 1), datetime.date(2021, 6, 10)])

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

#### Seasonality of temperature difference between river & GW. In summer, river is warmer than GW, and in winter, river is cooler than GW.

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

#### The storm seems to result in interesting delta temp difference between ParengaPl-river and EvansAve-river

## Storm impacts

#### River cooler than GW at this time. Therefore negative delta temp. The storm generated a larger difference in Parenga Pl compared to EvansAve.

In [ ]:
df3['ParengaPl_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - West 2 or Parenga (m)')
df3['EvansAve_dtemp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Δtemp River - East 3 of EvansAve (m)')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Δtemp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='SC at East 3 or EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 5, 29), datetime.date(2021, 6, 6)])

In [ ]:
df3['ParengaPl_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp ParengaPl')
df3['EvansAve_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp EvansAve')
df3['river_odyssey_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp river')
plt.legend(fontsize =20, loc=3)
plt.ylim(5,16)
plt.ylabel('temp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)
plt.ylim(17000, 19000)
plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

In [ ]:
df3['river_level_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Level river')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Level (Lyt m)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)
plt.ylim(17000, 19000)
plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

#### Increase in river level; salinization and then freshening of GW in Evans Ave by about 400 uS/cm. The GW temperature also reduced as a result of the storm. 

In [ ]:
df3['river_level_Lyt_m'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Level river')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Level (Lyt m)', fontsize=20)

df3.EvansAve_cond_uScm.plot(secondary_y = True, linewidth=5, fontsize=20, label='SC at EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)
plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

#### The reduction in GW temperature preceeds the reduction in river temp?? Rainfall effects too, not just river

In [ ]:
df3['river_odyssey_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='river temp')
df3['EvansAve_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='EvansAve temp')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Temp (C)', fontsize=20)
plt.ylim(5,16)

df3.EvansAve_cond_uScm.plot(secondary_y = True, linewidth=5, fontsize=20, label='SC at EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)
plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

In [ ]:
df3['river_odyssey_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='river temp')
df3['EvansAve_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='EvansAve temp')
plt.legend(fontsize =20, loc=3)
plt.ylabel('Temp (C)', fontsize=20)
plt.ylim(5,16)

df3.river_level_Lyt_m.plot(secondary_y = True, linewidth=5, fontsize=20, label='River level (m Lyt)')
plt.ylabel('River level (m Lyt)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)
plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

In [ ]:
df3['ParengaPl_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp West 2')
df3['EvansAve_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp East 3')
df3['river_odyssey_temp'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='temp river')
plt.legend(fontsize =20, loc=3)
plt.ylim(5,16)
plt.ylabel('temp (C)', fontsize=20)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', linewidth=5, fontsize=20, label='SC at West 2 or ParengaPl (μS/cm)')
df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'r', linewidth=5, fontsize=20, label='SC at East 3 or EvansAve (μS/cm)')
plt.ylabel('SC (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 5, 15), datetime.date(2021, 6, 10)])

In [ ]:
df3['OwlesTce_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Groundwater SC at West 1 (μS/cm)')

In [ ]:
import matplotlib.dates as mdates

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (uS/cm)')
plt.xlim([datetime.datetime(2021, 2, 4, 18), datetime.datetime(2021, 2, 6, 12)])
plt.legend(fontsize = 'xx-large', loc=1)

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\AvonRiver_SC_blip_downward.png')

In [ ]:
# How did upward blips impact the adjacent GW?
# PArenga Pl

df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (μS/cm)')
plt.ylabel('Avon River SC at Owles Jetty (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=1)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', fontsize=20, linewidth=5, label='Groundwater SC at West 2 (μS/cm)')
plt.ylim(16000,17500)
plt.ylabel('Groundwater SC at West 2 (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)


plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])
plt.xlabel('Date time (NZST)', fontsize=20)

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\AvonRiver_SC_blips_West2.png')

In [ ]:
# How did upward blips impact the adjacent GW?

# MarigoldLn or West1

df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (μS/cm)')
plt.ylabel('Avon River SC at Owles Jetty (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=1)

df3.MarigoldLn_cond_uScm.plot(secondary_y = True, style='g', fontsize=20, linewidth=5, label='Groundwater SC at West 1 (μS/cm)')
plt.ylim(15800,16000)
plt.ylabel('Groundwater SC at West 1 (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)


plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])
plt.xlabel('Date time (NZST)', fontsize=20)

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\AvonRiver_SC_blips_West1.png')

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (μS/cm)')
plt.ylabel('Avon River SC at Owles Jetty (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=1)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', fontsize=20, linewidth=5, label='Groundwater SC at West 2 (μS/cm)')
plt.ylim(16000,17500)
plt.ylabel('Groundwater SC at West 2 (μS/cm)', fontsize=20)
plt.legend(fontsize = 'xx-large', loc=4)


plt.xlim([datetime.date(2021, 2, 1), datetime.date(2021, 2, 4)])
plt.xlabel('Date time (NZST)', fontsize=20)

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\AvonRiver_SC_blips_West2.png')

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (uS/cm)')
plt.legend(fontsize = 'xx-large', loc=1)

df3.ParengaPl_cond_uScm.plot(secondary_y = True, style = 'g', fontsize=20)
plt.ylim(16000,17500)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 15), datetime.date(2021, 2, 18)])

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (uS/cm)')
plt.legend(fontsize = 'xx-large', loc=1)
plt.ylabel('Avon River specific conductance (μS/cm)', fontsize=20)

df3.river_level_Lyt_m.plot(secondary_y = True, style = 'g', fontsize=20, linewidth=5, label='Avon River level at Owles Jetty (m Lyttelton 1937)')
plt.legend(fontsize = 'xx-large', loc=4)
plt.ylim(-0.5,1.2)
plt.ylabel('Avon River level (m Lyttelton 1937)', fontsize=20)

plt.xlim([datetime.date(2021, 2, 16), datetime.date(2021, 2, 19)])

In [ ]:
df3['river_cond_uScm'].plot(figsize=(20,10), linewidth=5, fontsize=20, label='Avon River specific conductance (uS/cm)')
plt.legend(fontsize = 'xx-large', loc=1)

df3.EvansAve_cond_uScm.plot(secondary_y = True, style = 'g', fontsize=20)
plt.ylim(495,510)
plt.legend(fontsize = 'xx-large', loc=4)

plt.xlim([datetime.date(2021, 2, 15), datetime.date(2021, 2, 18)])

In [ ]:
plt.ylim(16000,17500)
plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Meeting notes\Figures\AvonRiver_SC_blips.png')

In [ ]:
table = pd.pivot_table(df3, values=['ParengaPl_cond_uScm', 'EvansAve_cond_uScm'], index=[datetime.date(2021, 2, 15), datetime.date(2021, 2, 18)],
                    aggfunc={'ParengaPl_cond_uScm': np.mean, 'EvansAve_cond_uScm': [min, max, np.mean]})

# Manual dips

In [ ]:
# Import Manual_dips CSV
Manual_dips = pd.read_csv('Manual_dips.csv')

# To datetime
Manual_dips['date_time_NZST']=pd.to_datetime(Manual_dips['date_time_NZST'], format = '%d/%m/%Y %H:%M')

# Set date_time as index
Manual_dips.set_index('date_time_NZST', inplace = True)

In [ ]:
Manual_dips

## APP192

In [ ]:
# Filter out manual dips at APP192
Manual_dips_APP192 = Manual_dips.loc[Manual_dips['well'] == 'APP192']

# Extract the date_time index as a list
Manual_dips_APP192_datetime = list(Manual_dips_APP192.index)

# Turn the date_time list into a dataframe
Manual_dips_APP192_datetime = pd.DataFrame(Manual_dips_APP192_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_APP192_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
APP192 = df3[['APP192_WL_TOC_m']].copy()

# Concat APP192 and Manual_dips_APP192_datetime
APP192_concat = pd.concat([APP192, Manual_dips_APP192_datetime])

# Interpolate NaN values using fifth order polynomial function
APP192_concat = APP192_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_APP192 = Manual_dips_APP192.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_APP192 = Manual_dips_APP192.merge(APP192_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_APP192["residual_m"] = Manual_dips_APP192["manual_WL_m_TOC"] - Manual_dips_APP192["APP192_WL_TOC_m"]

In [ ]:
# Calculate the Residual_column
Manual_dips_APP192["residual_squared"] = Manual_dips_APP192["residual_m"]**2

In [ ]:
Manual_dips_APP192

In [ ]:
np.sqrt((Manual_dips_APP192["residual_squared"].sum()-0.047304)/(3-2)) # Residual of standard deviation, excluding 25Nov dip

In [ ]:
Results = pd.DataFrame()
d = {'Well': 'APP192', 'Res_SD':np.sqrt((Manual_dips_APP192["residual_squared"].sum()-0.047304)/(3-2)), 'N': '3'}
Results = Results.append(d, ignore_index=True)
Results

In [ ]:
# Check why large residual 2020-11-25 08:55:00
APP192_25Nov = APP192.loc['2020-11-25 08:40:00':'2020-11-25 12:00:00']
APP192_25Nov
# The readings look normal, so could it be that the logger data is in NZDT?! More likely operator error? Ignore this manual reading.

In [ ]:
Manual_dips_APP192.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\APP192.csv')

## ParengaPl

In [ ]:
# Filter out manual dips at ParengaPl
Manual_dips_ParengaPl = Manual_dips.loc[Manual_dips['well'] == 'ParengaPl']

# Extract the date_time index as a list
Manual_dips_ParengaPl_datetime = list(Manual_dips_ParengaPl.index)

# Turn the date_time list into a dataframe
Manual_dips_ParengaPl_datetime = pd.DataFrame(Manual_dips_ParengaPl_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_ParengaPl_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
ParengaPl = df3[['ParengaPl_WL_TOC_m']].copy()

# Concat ParengaPl and Manual_dips_ParengaPl_datetime
ParengaPl_concat = pd.concat([ParengaPl, Manual_dips_ParengaPl_datetime])

# Interpolate NaN values using fifth order polynomial function
ParengaPl_concat = ParengaPl_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_ParengaPl = Manual_dips_ParengaPl.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_ParengaPl = Manual_dips_ParengaPl.merge(ParengaPl_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_ParengaPl["residual_m"] = Manual_dips_ParengaPl["manual_WL_m_TOC"] - Manual_dips_ParengaPl["ParengaPl_WL_TOC_m"]


In [ ]:
# Calculate the Residual_column
Manual_dips_ParengaPl["residual_squared"] = Manual_dips_ParengaPl["residual_m"]**2

In [ ]:
np.sqrt((Manual_dips_ParengaPl["residual_squared"].sum()-0.002420)/(5-2))
# Residual of standard deviation. Deleted a value bcs of double-up.

In [ ]:
d = {'Well': 'ParengaPl', 'Res_SD':np.sqrt((Manual_dips_ParengaPl["residual_squared"].sum()-0.002420)/(5-2)), 'N': '5'}
Results = Results.append(d, ignore_index=True)
Results

In [ ]:
Manual_dips_ParengaPl

In [ ]:
Manual_dips_ParengaPl.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\ParengaPl.csv')

In [ ]:
# Check why large residual 10 June 2021
ParengaPl_10June = ParengaPl.loc['2021-06-10 10:30:00':'2021-06-10 11:40:00']
ParengaPl_10June

In [ ]:
# Check why large residual 2020-11-25 09:51:00
ParengaPl_25Nov = ParengaPl.loc['2020-11-25 09:10:00':'2020-11-25 10:50:00']
ParengaPl_25Nov

In [ ]:
# Check why large residual 10:58, 11:30
ParengaPl_16Apr = ParengaPl.loc['2021-04-16 10:40:00':'2021-04-16 12:00:00']
ParengaPl_16Apr

## MarigoldLn

In [ ]:
# Filter out manual dips at MarigoldLn
Manual_dips_MarigoldLn = Manual_dips.loc[Manual_dips['well'] == 'MarigoldLn']

# Extract the date_time index as a list
Manual_dips_MarigoldLn_datetime = list(Manual_dips_MarigoldLn.index)

# Turn the date_time list into a dataframe
Manual_dips_MarigoldLn_datetime = pd.DataFrame(Manual_dips_MarigoldLn_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_MarigoldLn_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
MarigoldLn = df3[['MarigoldLn_WL_TOC_m']].copy()

# Concat MarigoldLn and Manual_dips_MarigoldLn_datetime
MarigoldLn_concat = pd.concat([MarigoldLn, Manual_dips_MarigoldLn_datetime])

# Interpolate NaN values using fifth order polynomial function
MarigoldLn_concat = MarigoldLn_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_MarigoldLn = Manual_dips_MarigoldLn.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_MarigoldLn = Manual_dips_MarigoldLn.merge(MarigoldLn_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_MarigoldLn["residual_m"] = Manual_dips_MarigoldLn["manual_WL_m_TOC"] - Manual_dips_MarigoldLn["MarigoldLn_WL_TOC_m"]

In [ ]:
# Calculate the Residual_column
Manual_dips_MarigoldLn["residual_squared"] = Manual_dips_MarigoldLn["residual_m"]**2

In [ ]:
Manual_dips_MarigoldLn

In [ ]:
np.sqrt((Manual_dips_MarigoldLn["residual_squared"].sum())/(5-2))

In [ ]:
d = {'Well': 'MarigoldLn', 'Res_SD':np.sqrt((Manual_dips_MarigoldLn["residual_squared"].sum())/(5-2)), 'N': '5'}
Results = Results.append(d, ignore_index=True)
Results

In [ ]:
Manual_dips_MarigoldLn.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\MarigoldLn.csv')

In [ ]:
# Check why large residual 16 April 2021
MarigoldLn_14Oct = MarigoldLn.loc['2020-10-14 12:30:00':'2020-10-14 13:10:00']
MarigoldLn_14Oct

#to be deleted 2020-10-14 12:50:00

## EvansAve

In [ ]:
# Filter out manual dips at EvansAve
Manual_dips_EvansAve = Manual_dips.loc[Manual_dips['well'] == 'EvansAve']

# Extract the date_time index as a list
Manual_dips_EvansAve_datetime = list(Manual_dips_EvansAve.index)

# Turn the date_time list into a dataframe
Manual_dips_EvansAve_datetime = pd.DataFrame(Manual_dips_EvansAve_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_EvansAve_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
EvansAve = df3[['EvansAve_WL_TOC_m']].copy()

# Concat EvansAve and Manual_dips_EvansAve_datetime
EvansAve_concat = pd.concat([EvansAve, Manual_dips_EvansAve_datetime])

# Interpolate NaN values using fifth order polynomial function
EvansAve_concat = EvansAve_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_EvansAve = Manual_dips_EvansAve.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_EvansAve = Manual_dips_EvansAve.merge(EvansAve_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_EvansAve["residual_m"] = Manual_dips_EvansAve["manual_WL_m_TOC"] - Manual_dips_EvansAve["EvansAve_WL_TOC_m"]

In [ ]:
# Calculate the Residual_column
Manual_dips_EvansAve["residual_squared"] = Manual_dips_EvansAve["residual_m"]**2

In [ ]:
Manual_dips_EvansAve

In [ ]:
np.sqrt((Manual_dips_EvansAve["residual_squared"].sum()-0.000252-0.003375)/(5-2))

In [ ]:
d = {'Well': 'EvansAve', 'Res_SD':np.sqrt((Manual_dips_EvansAve["residual_squared"].sum()-0.000252-0.003375)/(5-2)), 'N': '5'}
Results = Results.append(d, ignore_index=True)
Results

In [ ]:
Manual_dips_EvansAve.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\EvansAve.csv')

In [ ]:
# Check why large residual 18 Jan 2021
EvansAve_18Jan = EvansAve.loc['2021-01-18 11:40:00':'2021-01-18 12:30:00']
EvansAve_18Jan

#Evans Ave Delete erroneous reading 2020-10-14 10:50:00, 2020-10-14 11:00:00, 2020-10-14 11:10:00, 2020-11-25 08:30:00
#Locate datetime 2020-10-14 10:31:25 before merging; which dataset creates this error; could be river loggers?

In [ ]:
# Check why large residual 16 April 2021
EvansAve_25Nov = EvansAve.loc['2020-11-25 08:00:00':'2020-11-25 09:00:00']
EvansAve_25Nov

#Delete erroneous reading 2020-11-25 08:30:00

## OwlesTce

In [ ]:
# Filter out manual dips at OwlesTce
Manual_dips_OwlesTce = Manual_dips.loc[Manual_dips['well'] == 'OwlesTce']

# Extract the date_time index as a list
Manual_dips_OwlesTce_datetime = list(Manual_dips_OwlesTce.index)

# Turn the date_time list into a dataframe
Manual_dips_OwlesTce_datetime = pd.DataFrame(Manual_dips_OwlesTce_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_OwlesTce_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
OwlesTce = df3[['OwlesTce_WL_TOC_m']].copy()

# Concat OwlesTce and Manual_dips_OwlesTce_datetime
OwlesTce_concat = pd.concat([OwlesTce, Manual_dips_OwlesTce_datetime])

# Interpolate NaN values using fifth order polynomial function
OwlesTce_concat = OwlesTce_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_OwlesTce = Manual_dips_OwlesTce.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_OwlesTce = Manual_dips_OwlesTce.merge(OwlesTce_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_OwlesTce["residual_m"] = Manual_dips_OwlesTce["manual_WL_m_TOC"] - Manual_dips_OwlesTce["OwlesTce_WL_TOC_m"]


In [ ]:
# Calculate the Residual_column
Manual_dips_OwlesTce["residual_squared"] = Manual_dips_OwlesTce["residual_m"]**2

In [ ]:
Manual_dips_OwlesTce

# Why is there a double-up in the 18 Jan datetime?

In [ ]:
np.sqrt((Manual_dips_OwlesTce["residual_squared"].sum()-0.014139*2)/(3-2))

In [ ]:
d = {'Well': 'OwlesTce', 'Res_SD':np.sqrt((Manual_dips_OwlesTce["residual_squared"].sum()-0.014139*2)/(3-2)), 'N': '3'}
Results = Results.append(d, ignore_index=True)
Results

In [ ]:
Results.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\Res_SD_results.csv')

In [ ]:
Manual_dips_OwlesTce.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\OwlesTce.csv')

In [ ]:
result = pd.concat([Manual_dips_OwlesTce, Manual_dips_APP192, Manual_dips_MarigoldLn, Manual_dips_ParengaPl, Manual_dips_EvansAve])
result.to_csv(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Timeseries\out\Manual_dip_residual\result.csv')

In [ ]:
# Check why large residual 18 Jan 2021
OwlesTce_25Nov = OwlesTce.loc['2020-11-25 08:40:00':'2020-11-25 09:30:00']
OwlesTce_25Nov

# Residual of 0.1 m is rather high? Could be operator error?

#2020-11-25 09:10:00 error

In [ ]:
# Check why large residual 10 June 2021
OwlesTce_10June = OwlesTce.loc['2021-06-10 13:40:00':'2021-06-10 14:20:00']
OwlesTce_10June

# Delete erroneous reading 2021-06-10 14:00:00

In [ ]:
import datetime
# Check river calc
df3.loc[datetime.datetime(2021, 1, 18, 13, 0):datetime.datetime(2021, 1, 18, 13, 40), 'baro_pressure_kPa']

In [ ]:
(99.2661-99.1953)* 0.101972 #Baro at OwlesTce-Baro at Brooker

## APP165

In [ ]:
# Filter out manual dips at APP165
Manual_dips_APP165 = Manual_dips.loc[Manual_dips['well'] == 'APP165']

# Extract the date_time index as a list
Manual_dips_APP165_datetime = list(Manual_dips_APP165.index)

# Turn the date_time list into a dataframe
Manual_dips_APP165_datetime = pd.DataFrame(Manual_dips_APP165_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_APP165_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
APP165 = df3[['APP165_WL_TOC_m']].copy()

# Concat APP165 and Manual_dips_APP165_datetime
APP165_concat = pd.concat([APP165, Manual_dips_APP165_datetime])

# Interpolate NaN values using fifth order polynomial function
APP165_concat = APP165_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_APP165 = Manual_dips_APP165.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_APP165 = Manual_dips_APP165.merge(APP165_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_APP165["residual_m"] = Manual_dips_APP165["manual_WL_m_TOC"] - Manual_dips_APP165["APP165_WL_TOC_m"]

In [ ]:
Manual_dips_APP165

In [ ]:
# Check why large residual 16 April 2021
APP165_16April = APP165.loc['2021-04-16 12:30:00':'2021-04-16 13:30:00']
APP165_16April

In [ ]:
# Check why large residual 16 April 2021
APP165_concat_16April = APP165_concat.loc['2021-04-16 12:30:00':'2021-04-16 13:30:00']
APP165_concat_16April

# Why does APP165 has such large residual?
# NEED FURTHER INVESTIGATION!

## APP103

In [ ]:
# Filter out manual dips at APP103
Manual_dips_APP103 = Manual_dips.loc[Manual_dips['well'] == 'APP103']

# Extract the date_time index as a list
Manual_dips_APP103_datetime = list(Manual_dips_APP103.index)

# Turn the date_time list into a dataframe
Manual_dips_APP103_datetime = pd.DataFrame(Manual_dips_APP103_datetime, columns =['date_time'])

# Set date_time as index
Manual_dips_APP103_datetime.set_index('date_time', inplace = True)

# Create new dataframe for specific wells
APP103 = df3[['APP103_WL_TOC_m']].copy()

# Concat APP103 and Manual_dips_APP103_datetime
APP103_concat = pd.concat([APP103, Manual_dips_APP103_datetime])

# Interpolate NaN values using fifth order polynomial function
APP103_concat = APP103_concat.interpolate(method='polynomial', order=5)

# Rename the column name because it was giving errors
Manual_dips_APP103 = Manual_dips_APP103.rename(columns={"date_time_NZST": "date_time"})

# Merge df_merge and the Comparison dataframe
Manual_dips_APP103 = Manual_dips_APP103.merge(APP103_concat, how='inner', left_index=True, right_index=True)

# Calculate the Residual_column
Manual_dips_APP103["residual_m"] = Manual_dips_APP103["manual_WL_m_TOC"] - Manual_dips_APP103["APP103_WL_TOC_m"]

In [ ]:
Manual_dips_APP103

# Sum monthly qx

In [ ]:
filtered_df = pd.DataFrame()
volume=pd.DataFrame()

for col in df3.columns:
    if "_vol_m3/m2" in col:
        filtered_df[col] = df3[col]

In [ ]:
Jan=filtered_df.loc[datetime.datetime(2021, 1, 1, 0, 0):datetime.datetime(2021, 1, 31, 23, 50)]
Feb=filtered_df.loc[datetime.datetime(2021, 2, 1, 0, 0):datetime.datetime(2021, 2, 28, 23, 50)]
Mar=filtered_df.loc[datetime.datetime(2021, 3, 1, 0, 0):datetime.datetime(2021, 3, 31, 23, 50)]
Apr=filtered_df.loc[datetime.datetime(2021, 4, 1, 0, 0):datetime.datetime(2021, 4, 30, 23, 50)]
May=filtered_df.loc[datetime.datetime(2021, 5, 1, 0, 0):datetime.datetime(2021, 5, 31, 23, 50)]

wells = os.listdir(raw_well_data_directory)
well_dataframes = []
for well in wells:
        d = {'Well': f"{well}", 'Sum_vol_m3/m2':Jan[f"{well}_vol_m3/m2"].sum(), 'Month':"Jan", 'date_time_start': Jan.index.min(), 'date_time_end': Jan.index.max()}
        volume = volume.append(d, ignore_index=True)
        d = {'Well': f"{well}", 'Sum_vol_m3/m2':Feb[f"{well}_vol_m3/m2"].sum(), 'Month':"Feb", 'date_time_start': Feb.index.min(), 'date_time_end': Feb.index.max()}
        volume = volume.append(d, ignore_index=True)
        d = {'Well': f"{well}", 'Sum_vol_m3/m2':Mar[f"{well}_vol_m3/m2"].sum(), 'Month':"Mar", 'date_time_start': Mar.index.min(), 'date_time_end': Mar.index.max()}
        volume = volume.append(d, ignore_index=True)
        d = {'Well': f"{well}", 'Sum_vol_m3/m2':Apr[f"{well}_vol_m3/m2"].sum(), 'Month':"Apr", 'date_time_start': Apr.index.min(), 'date_time_end': Apr.index.max()}
        volume = volume.append(d, ignore_index=True)
        d = {'Well': f"{well}", 'Sum_vol_m3/m2':May[f"{well}_vol_m3/m2"].sum(), 'Month':"May", 'date_time_start': May.index.min(), 'date_time_end': May.index.max()}
        volume = volume.append(d, ignore_index=True)

        

del Jan, Feb, Mar, Apr, May
volume

In [ ]:
volume.to_csv(os.path.relpath('../out/Volume_sum_monthly.csv'))
del volume

# SC peak analysis

In [ ]:
import datetime

In [ ]:
# Max SC date time (river SC)

df_loc=df3.loc[datetime.datetime(2021, 2, 1, 0, 0):datetime.datetime(2021, 2, 1, 12, 0), 'river_cond_uScm']
df_loc

In [ ]:
df_loc.max()

In [ ]:
df_loc.idxmax()

In [ ]:
df4=df3.loc[datetime.datetime(2021, 2, 1, 9, 20):datetime.datetime(2021, 2, 2, 0, 0)]

interesting_df = pd.DataFrame()

for col in df4.columns:
    if "_cond_uScm" in col:
        interesting_df[col] = df4[col]

interesting_df

In [ ]:
Results = pd.DataFrame()

In [ ]:
wells = os.listdir(raw_well_data_directory)
well_dataframes = []
for well in wells:
        d = {'Site': f"{well}", 'Date_time_max_SC':interesting_df[f"{well}_cond_uScm"].idxmax(), 'Time_diff_river_peak':interesting_df[f"{well}_cond_uScm"].idxmax() - df_loc.idxmax()}
        Results = Results.append(d, ignore_index=True)
        
Results

In [ ]:
Results.to_csv(os.path.relpath('../out/SC_peak_analysis_20210201.csv'))
# del Results

In [ ]:
# Transect 2 with velocity subplot

datetime_start = datetime.datetime(2021, 2, 1, 0, 0)
datetime_end = datetime.datetime(2021, 2, 2, 0, 0)

x = df3.index


# GW SC
# East in ax1
y1 = df3['OwlesTce_cond_uScm']
y2 = df3['APP192_cond_uScm']

# West in twin1
y3 = df3['MarigoldLn_cond_uScm']

# River SC in twin 2
y4 = df3['river_cond_uScm']

# River & GW hf in ax2
y5 = df3['river_hf_Lyt_m']

y5_upper = df3['river_hf_Lyt_m'] + df3['river_ohf']
y5_lower = df3['river_hf_Lyt_m'] - df3['river_ohf']

y6 = df3['OwlesTce_hf_Lyt_m']

y6_upper = df3['OwlesTce_hf_Lyt_m'] + df3['OwlesTce_ohf']
y6_lower = df3['OwlesTce_hf_Lyt_m'] - df3['OwlesTce_ohf']

y7 = df3['APP192_hf_Lyt_m']

y7_upper = df3['APP192_hf_Lyt_m'] + df3['APP192_ohf']
y7_lower = df3['APP192_hf_Lyt_m'] - df3['APP192_ohf']

y8 = df3['MarigoldLn_hf_Lyt_m']

y8_upper = df3['MarigoldLn_hf_Lyt_m'] + df3['MarigoldLn_ohf']
y8_lower = df3['MarigoldLn_hf_Lyt_m'] - df3['MarigoldLn_ohf']

# Velocity middle subplot
y9= df3['OwlesTce_river_q_md']
y10= df3['APP192_river_q_md']
y11= df3['MarigoldLn_river_q_md']

y9_upper = df3['OwlesTce_river_q_md'] + df3['OwlesTce_qx_SD']
y9_lower = df3['OwlesTce_river_q_md'] - df3['OwlesTce_qx_SD']

y10_upper = df3['APP192_river_q_md'] + df3['APP192_qx_SD']
y10_lower = df3['APP192_river_q_md'] - df3['APP192_qx_SD']

y11_upper = df3['MarigoldLn_river_q_md'] + df3['MarigoldLn_qx_SD']
y11_lower = df3['MarigoldLn_river_q_md'] - df3['MarigoldLn_qx_SD']

fig, (ax2, ax1, ax3) = plt.subplots(3)
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(6.6)

twin1 = ax1.twinx()
twin2 = ax1.twinx()

# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines['right'].set_position(("axes", 1.16))

# GW SC plot
ax1.plot(x, y1, color=OwlesTce_SC_col, label="1-L1")
ax1.plot(x, y2, color=APP192_SC_col, label="1-L2")
twin1.plot(x, y3, color=MarigoldLn_SC_col, label="1-R")

# River SC plot
twin2.plot(x, y4, color=River_SC_col, label="River")

# Heads plot
ax2.plot(x, y5, color=River_SC_col, label="River")
ax2.plot(x, y6, color=OwlesTce_SC_col, label="1-L1")
ax2.plot(x, y7, color=APP192_SC_col, label="1-L2")
ax2.plot(x, y8, color=MarigoldLn_SC_col, label="1-R")

ax3.plot(x, y9, color=OwlesTce_SC_col, label="1-L1")
ax3.plot(x, y10, color=APP192_SC_col, label="1-L2")
ax3.plot(x, y11, color=MarigoldLn_SC_col, label="1-R")

ax3.set_xlabel("Date time")
ax1.set_xlim([datetime_start, datetime_end])
ax1.set_ylim(1500,2600) # y limits for east GW SC (ax1)
twin1.set_ylim(15800,16100) # y limits for West GW SC (twina1)
ax2.set_ylim(-0.4,1.5)

# Format date in x ax1is
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax3.xaxis.set_major_formatter(myFmt)

#plt.setp(ax3.get_xticklabels(), visible=True, rotation=30, ha='right')

# Rotate tick labels to 45 degrees
for tick in ax3.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax1.set_ylabel(u"True left groundwater SC\n(\u03bcScm$^{-1}$)")
twin1.set_ylabel(u"True right groundwater SC\n(\u03bcScm$^{-1}$)")
twin2.set_ylabel(u"River SC (\u03bcScm$^{-1}$)")

twin2.yaxis.label.set_color(River_SC_col)

tkw = dict(size=4, width=1.5)

ax1.tick_params(axis='y', **tkw)
ax1.tick_params(axis='x', **tkw)

twin1.tick_params(axis='y', **tkw)

twin2.tick_params(axis='y', colors=River_SC_col, **tkw)

ax2.fill_between(x, y5_upper, y5_lower, alpha=0.2)
ax2.fill_between(x, y6_upper, y6_lower, color=OwlesTce_SC_col, alpha=0.2)
ax2.fill_between(x, y7_upper, y7_lower, color=APP192_SC_col, alpha=0.2)
ax2.fill_between(x, y8_upper, y8_lower, color=MarigoldLn_SC_col, alpha=0.2)

datetime_FB_start=datetime.datetime(2021, 2, 1, 0, 0)
datetime_FB_end=datetime.datetime(2021, 2, 1, 23, 0)

ax3.fill_between(x, y9_upper, y9_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=OwlesTce_SC_col, alpha=0.2)
ax3.fill_between(x, y10_upper, y10_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=APP192_SC_col, alpha=0.2)
ax3.fill_between(x, y11_upper, y11_lower,
                 where = (x >= datetime_FB_start) & (x <= datetime_FB_end),
                 color=MarigoldLn_SC_col, alpha=0.2)

ax2.set_ylabel('Freshwater head, $h_f$\n(m LVD)')
ax2.set_xlim([datetime_start, datetime_end])

# Remove x axis labels for ax1 and ax2
ax2.set_xticklabels([])
ax1.set_xticklabels([])

# Legend ax2
ax2.legend(bbox_to_anchor=(1.01,1.05), loc="upper left")

ax3.set_ylabel('Horizontal flux, $q_x$\n($\mathregular{m^{3}m^{-2}day^{-1}}$)')
ax3.set_xlim([datetime_start, datetime_end])
ax3.set_ylim(-1.5,1.4)
ax3.axhline(y=0, color='#c7c7c7', linestyle='--')  #horizontal line

ax2.text(datetime_start + datetime.timedelta(hours=1), 1.2,'a',fontsize=16,fontweight="bold")
ax1.text(datetime_start + datetime.timedelta(hours=1), 2400,'b',fontsize=16,fontweight="bold")
ax3.text(datetime_start + datetime.timedelta(hours=1), 0.9,'c',fontsize=16,fontweight="bold")

plt.style.use('default')

plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\T1_3plots_20210201_20210204_dry_errorshade_flux_20220428_1552.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y3, y4, y5, y6, y8, y6_upper, y6_lower, y8_upper, y8_lower, y5_upper, y5_lower, y9_upper, y9_lower, y10_upper, y10_lower, y11_upper, y11_lower, datetime_start, datetime_end, datetime_FB_start, datetime_FB_end


In [ ]:
# Max sea level date time

Tide_loc=Tide.loc[datetime.datetime(2021, 2, 1, 0, 0):datetime.datetime(2021, 2, 1, 12, 0), 'sea_level_m_Lyt']
Tide_loc

In [ ]:
print("Max sea level (first tidal cycle 1 Feb 2021): ", Tide_loc.max())
print("Date time of max sea level: " , Tide_loc.idxmax())
print("Time difference sea level peak - river level peak 1 Feb 2021: ", df_loc.idxmax()-Tide_loc.idxmax())

In [ ]:
datetime_start = datetime.date(2021, 2, 1)
datetime_end = datetime.date(2021, 2, 2)


# Define x and y
x = df3.index
y1 = df3['river_level_Lyt_m']
y2 = df3['river_cond_uScm']
x3 = Tide.index
y3 = Tide['sea_level_m_Lyt']

fig, ax = plt.subplots()
fig.subplots_adjust(right=0.75)
fig.set_figwidth(10)
fig.set_figheight(3)

twin1 = ax.twinx()

p1, = ax.plot(x, y1, color=River_SC_col, label="River level")
p2, = ax.plot(x3, y3, color='#83d982', label="Sea level")
p3, = twin1.plot(x, y2, color='#D81B60', label="River SC")

ax.set_xlabel("Date time")
ax.set_xlim([datetime_start, datetime_end])

# Format date in x axis
myFmt = mdates.DateFormatter('%d %b %y %H:%M')
ax.xaxis.set_major_formatter(myFmt)

# Rotate tick labels to 45 degrees
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    tick.set_horizontalalignment('right')

ax.set_ylabel("Level (m LVD)")
twin1.set_ylabel(u"River SC (\u03bcS/cm)")

#ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors='black', **tkw)
twin1.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

ax.legend(handles=[p1, p2, p3], loc=1, ncol=3)


# plt.rcParams["figure.figsize"] = (20,5)
# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

#plt.savefig(r'C:\Users\setiawai\OneDrive - Lincoln University\PhD\Thesis\Field_investigation_manuscript\Figures\River_sea_levels_SC_20210201_20210209_v3.png', dpi=300, bbox_inches="tight")

plt.show()

del x, y1, y2, x3, y3, datetime_start, datetime_end